In [1]:
from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn, join, vstack
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io.votable import parse
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
# Suppress SettingWithCopyWarnings
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from tqdm import tqdm

#to query Gaia
from astroquery.gaia import Gaia

#to import SDSS-IV data
from astroquery.sdss import SDSS

#for multiprocessing
from joblib import Parallel, delayed

In [2]:
MsunCGS=1.989*10**33 #g
RsunCGS=6.955*10**10 #cm
cCGS=2.99792458*10**10 #cm/s
GCGS=6.6743*10**(-8) #cm^3 g^-1 s^-2

# This notebook gets the sample of all earlier SDSS spectra flagged as DA WDs

# Download Nicola's list of historic SDSS WDs from DRs 1-16

Catalogs from the NGF21 paper are available at https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/508/3877#/browse

The list of historic SDSS WDs is the sdssspec.dat.gz file

- GaiaEDR3  Gaia EDR3 source_id (source_id)
- Plate     Identifier of the plate used in the observation of the spectrum
- MJD       Modified Julian date of the observation of the spectrum
- FiberID   Identifier of the fiber used in the observation of the spectrum
- S/N       Signal-to-noise ratio of the spectrum calculated in the range 4500-5500{AA}
- specClass Classification of the object based on a visual inspection of the SDSS spectrum (spec_class)
- RAdeg     Gaia EDR3 Barycentric right ascension (ICRS) at Ep=2016.0 (ra)
- DEdeg     Gaia EDR3 Barycentric declination (ICRS) at Ep=2016.0 (dec)
- TeffH     ?=- Effective temperature from fitting the dereddened G, BP, and RP absolute fluxes with pure-H model atmospheres ((teff_H)
- e_TeffH     ?=- Uncertainty on TeffH (eteff_H)
- loggH     ?=- Log of surface gravity from fitting the dereddened G, BP, and RP absolute fluxes with pure-H model atmospheres ((logg_H)
- e_loggH     ?=- Uncertainty one loggH (elogg_H)
- MassH     ?=- Stellar mass resulting from the adopted mass-radius relationand best fit parameters (mass_H)
- e_MassH     ?=- Uncertainty on MassH (emass_H)

My initial list of DR 1-14 DA WDs included $\sim$6000 additional spectra. Nicola reinspected these spectra so some DA spectra were lost to reclassification ($\sim$500-600). Additionally, some spectra were not included in his list due to not having a reliable Gaia match because the WD was too faint ($\sim$3000), some had negative parallaxes or parallax/error<1 so had no reliable distance measurement ($\sim$1000).  Another $\sim$1000 were rejected for other reasons such as bad photometry. 

Note that all measured parameters are measured with Gaia photometry. We will also want to cross match with the published DRs 1-14 parameters to get spectroscopic measurements for comparison.

In [3]:
gaia_df = pd.read_csv('data/J_MNRAS_508_3877_sdssspec.dat.gz.txt',sep='|')
#drop the first row of dashes and the last row of NaNs
gaia_df=gaia_df.drop(0)
gaia_df=gaia_df.dropna()
#remove spaces from column names
gaia_df.columns=gaia_df.columns.str.replace(' ', '')
#keep only the relevant columns
gaia_df=gaia_df[['GaiaEDR3','WDcatt','Plate','MJD','FiberID','RAdegDEdeg','S/N','specClass','TeffH','e_TeffH','loggH','e_loggH','MassH','e_MassH']]
#break the RA and Dec column into 2 columns
ras=[]
decs=[]
for i in range(len(gaia_df)):
    ra=gaia_df['RAdegDEdeg'][i+1][:15]
    dec=gaia_df['RAdegDEdeg'][i+1][15:]
    ras.append(float(ra))
    decs.append(float(dec))
gaia_df['ra']=ras
gaia_df['dec']=decs
#rename to GAIA_DR3_SOURCE_ID
gaia_df=gaia_df.rename(columns={"GaiaEDR3": "gaia_edr3_source_id","Plate": "plate","MJD": "mjd","FiberID": "fiber",
                                "S/N": "snr_gentile","TeffH": "teff_gentile","e_TeffH": "e_teff_gentile",
                                "loggH": "logg_gentile","e_loggH": "e_logg_gentile","MassH":"mass_gentile",
                               "e_MassH":"e_mass_gentile"})
#set rows where WD parameters couldn't be measured to 0
gaia_df=gaia_df.replace(['      ---','     ---'],0)
#convert all columns to numeric values
gaia_df['snr_gentile']=pd.to_numeric(gaia_df['snr_gentile'])
gaia_df['teff_gentile']=pd.to_numeric(gaia_df['teff_gentile'])
gaia_df['e_teff_gentile']=pd.to_numeric(gaia_df['e_teff_gentile'])
gaia_df['logg_gentile']=pd.to_numeric(gaia_df['logg_gentile'])
gaia_df['e_logg_gentile']=pd.to_numeric(gaia_df['e_logg_gentile'])
gaia_df['mass_gentile']=pd.to_numeric(gaia_df['mass_gentile'])
gaia_df['e_mass_gentile']=pd.to_numeric(gaia_df['e_mass_gentile'])
#convert gaia_dr3_source_id and pmf columns to integers
gaia_df['gaia_edr3_source_id']=gaia_df['gaia_edr3_source_id'].astype(int)
gaia_df['plate']=gaia_df['plate'].astype(int)
gaia_df['mjd']=gaia_df['mjd'].astype(int)
gaia_df['fiber']=gaia_df['fiber'].astype(int)
#add unique PMF identifyer
gaia_df['pmf']=gaia_df['plate'].astype(str)+gaia_df['mjd'].astype(str)+gaia_df['fiber'].astype(str)
gaia_df['pmf']=gaia_df['pmf'].astype(int)
#reorder columns and drop RAdegDEdeg column
cols=['gaia_edr3_source_id', 'WDcatt','plate', 'mjd', 'fiber','pmf','ra', 'dec','snr_gentile', 'specClass', 
      'teff_gentile', 'e_teff_gentile','logg_gentile', 'e_logg_gentile', 'mass_gentile', 'e_mass_gentile']
gaia_df = gaia_df[cols]
#calculate the radius in solar units
gaia_df['radius_gentile']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*gaia_df['mass_gentile']/10**gaia_df['logg_gentile'])
#calculate errors on the radius
gaia_df['e_radius_gentile']=gaia_df['radius_gentile']*np.sqrt(((1/(4*(gaia_df['mass_gentile'])**2))*(gaia_df['e_mass_gentile'])**2)+((1/4)*((np.log(10))**2)*(gaia_df['e_logg_gentile'])**2))


/var/folders/ny/bjybyvhj6s9_2ms5cmld1xbc0000gn/T/ipykernel_1808/2996969217.py:1: DtypeWarning: Columns (1,5,6,9,10,11,12,13,14,37,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  gaia_df = pd.read_csv('data/J_MNRAS_508_3877_sdssspec.dat.gz.txt',sep='|')


In [4]:
#get only DAs
eSDSS_DA_df=gaia_df.query('specClass=="DA"')
#reset the indices
indices=np.arange(0,len(eSDSS_DA_df))
eSDSS_DA_df.set_index(indices,inplace=True)
display(eSDSS_DA_df)
print(len(eSDSS_DA_df['gaia_edr3_source_id'].unique()))

,gaia_edr3_source_id,WDcatt,plate,mjd,fiber,pmf,ra,dec,snr_gentile,specClass,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile
0,2449845669146456576,M,7850,56956,719,785056956719,0.028157,-0.781642,23.0,DA,11924.81,1437.04,8.168734,0.258491,0.708278,0.160616,0.011480,0.003656
1,2449845669146456576,M,685,52203,225,68552203225,0.028157,-0.781642,14.0,DA,11924.81,1437.04,8.168734,0.258491,0.708278,0.160616,0.011480,0.003656
2,2422931587299822848,M,650,52143,497,65052143497,0.029901,-9.727937,7.0,DA,18893.32,8909.88,7.039190,1.315437,0.288982,0.267948,0.026919,0.042635
3,2746738620674124928,M,11279,58449,954,1127958449954,0.030335,7.829031,7.0,DA,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,2873402974373195008,M,7134,56566,587,713456566587,0.032837,30.768414,12.0,DA,8654.57,914.01,7.836862,0.362041,0.504615,0.183625,0.014199,0.006458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25410,2738328623735088128,M,9179,57682,11,91795768211,359.961198,1.127805,26.0,DA,10005.75,691.56,7.878053,0.235699,0.531095,0.121613,0.013893,0.004092
25411,2738328623735088128,M,7848,56959,957,784856959957,359.961198,1.127805,38.0,DA,10005.75,691.56,7.878053,0.235699,0.531095,0.121613,0.013893,0.004092
25412,2854826003866560512,M,2824,54452,255,282454452255,359.973033,28.028322,18.0,DA,19326.00,4284.75,7.528121,0.458234,0.426229,0.162863,0.018620,0.010448
25413,2448241997078045696,M,4354,55810,348,435455810348,359.980211,-2.778191,6.0,DA,7440.26,1048.73,8.666441,0.407053,1.020691,0.231543,0.007771,0.003747


19256


### Cross-match with Gaia DR3 using the EDR3 RAs and Decs from Nicola's catalog

Nicola's ids are for EDR3 not DR3 and so some objects fall out of a source id cross match. 

- Go to https://gea.esac.esa.int/archive/
- Select Search>ADQL>Upload User Table (click on stack of disks with arrow pointing up)
- Upload table created below
    - Follow https://www.cosmos.esa.int/web/gaia-users/archive/collaborate#UploadUserTable to update the table metadata to set the RA and Dec columns
    - The table metadata can be modified using the "edit table" button that is encompassed by the red circle
- Then select Cross-match tables (click on stars) and cross match uploaded table with gaiadr3.gaia_source for radius=3.0 arcseconds
    - see section 2.3 at https://www.cosmos.esa.int/web/gaia-users/archive/combine-with-other-data
    - click show join query
    - edit the job name and query to have only the columns you need
        - 'separation', 'source_id', 'pmra','pmra_error', 'pmdec', 'pmdec_error', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_mag', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_mag', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_mag', 'phot_bp_rp_excess_factor', 'l', 'b'
    - click submit query
    - download the resulting file
    
https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html
- phot_g_mean_flux : G-band mean flux (double, Flux[e-/s]), Mean flux in the G-band. 
- phot_g_mean_flux_error : Error on G-band mean flux (double, Flux[e-/s]), Standard deviation of the G-band fluxes divided by sqrt(phot_g_n_obs) 
- phot_g_mean_mag : G-band mean magnitude (float, Magnitude[mag]), Mean magnitude in the G band. This is computed from the G-band mean flux applying the magnitude zero-point in the Vega scale. No error is provided for this quantity as the error distribution is only symmetric in flux space. This converts to an asymmetric error distribution in magnitude space which cannot be represented by a single error value. 
- Have same params as above for BP and RP bands
- phot_bp_rp_excess_factor : BP/RP excess factor (float), BP/RP excess factor estimated from the comparison of the sum of integrated BP and RP fluxes with respect to the flux in the G band. This measures the excess of flux in the BP and RP integrated photometry with respect to the G band. This excess is believed to be caused by background and contamination issues affecting the BP and RP data. Therefore a large value of this factor for a given source indicates systematic errors in the BP and RP photometry.
- bp_rp : BP - RP colour (float, Magnitude[mag]), BP - RP colour: phot_bp_mean_mag - phot_rp_mean_mag 
- bp_g : BP - G colour (float, Magnitude[mag]), BP - G colour: phot_bp_mean_mag - phot_g_mean_mag 
- g_rp : G - RP colour (float, Magnitude[mag]), G - RP colour: phot_g_mean_mag - phot_rp_mean_mag 
- l : Galactic longitude (double, Angle[deg]), Galactic Longitude of the object at reference epoch ref_epoch, see Section 3.1.7 of the release documentation for conversion details. 
- b : Galactic latitude (double, Angle[deg]), Galactic Latitude of the object at reference epoch ref_epoch, see Section 3.1.7 of the release documentation for conversion details. 
- - pmra : Proper motion in right ascension direction (double, Angular Velocity[mas/year]). Proper motion in right ascension of the source in ICRS at the reference epoch ref_epoch. This is the local tangent plane projection of the proper motion vector in the direction of increasing right ascension. 
- pmra_error : Standard error of proper motion in right ascension direction (double, Angular Velocity[mas/year] ). 
- pmdec : Proper motion in declination direction (double, Angular Velocity[mas/year] ). Proper motion in declination of the source at the reference epoch ref_epoch. This is the projection of the proper motion vector in the direction of increasing declination.
- pmdec_error : Standard error of proper motion in declination direction (double, Angular Velocity[mas/year] )

In [5]:
#create and save a table to upload to Gaia
Gaia_upload=eSDSS_DA_df[['ra','dec']]
Gaia_upload.to_csv('csv/Gaia_upload.csv',header=False)


In [7]:
#read in the cross match file
Gaia_votable = parse("csv/Gaiaresult.vot").get_first_table().to_table()
#convert to pandas
Gaia_df=Gaia_votable.to_pandas()
#sort by separation and drop duplicate matches, keeping only the nearest match
Gaia_df=Gaia_df.sort_values(by=['separation'])
Gaia_df=Gaia_df.drop_duplicates(subset='my_id',keep='first')
#set my id as the index
Gaia_df=Gaia_df.set_index('my_id')
#rename columns
Gaia_df=Gaia_df.rename(columns={"separation":"gaia_dr3_source_separation","SOURCE_ID":"gaia_dr3_source_id"})
#drop extra columns
Gaia_df=Gaia_df.drop(['RA','DEC','ra','dec','parallax','parallax_error'],axis=1)
print(len(eSDSS_DA_df))
#merge with SDSS DA dataframe on index
eSDSS_DA_df=pd.merge(left=Gaia_df,right=eSDSS_DA_df,left_index=True,right_index=True,how="inner")
print(len(eSDSS_DA_df))
#reorder columns and drop extra columns
cols=['gaia_dr3_source_id', 'plate', 'mjd', 'fiber', 'pmf', 'ra', 'dec','pmra','pmra_error', 'pmdec', 'pmdec_error',
      'phot_g_mean_flux','phot_g_mean_flux_error', 'phot_g_mean_mag', 'phot_bp_mean_flux','phot_bp_mean_flux_error',
      'phot_bp_mean_mag', 'phot_rp_mean_flux','phot_rp_mean_flux_error', 'phot_rp_mean_mag','phot_bp_rp_excess_factor',
      'l', 'b','snr_gentile','teff_gentile', 'e_teff_gentile', 'logg_gentile', 'e_logg_gentile','mass_gentile',
      'e_mass_gentile', 'radius_gentile', 'e_radius_gentile']
eSDSS_DA_df = eSDSS_DA_df[cols]
#reset the indices
indices=np.arange(0,len(eSDSS_DA_df))
eSDSS_DA_df.set_index(indices,inplace=True)

25415
25415


In [9]:
#record which WDs are in Gaia but do not have complete Gaia photometry
nophotbp=np.isnan(eSDSS_DA_df['phot_bp_mean_flux'].to_list())
nophotrp=np.isnan(eSDSS_DA_df['phot_rp_mean_flux'].to_list())
nogaiaphot=[]
for i in range(len(nophotbp)):
    if nophotbp[i]==False and nophotrp[i]==False:
        nogaiaphot.append(False)
    else:
        nogaiaphot.append(True)
eSDSS_DA_df['no_gaia_phot']=nogaiaphot

# For each DA spectrum from Nicola's list, cross match with the SDSS DR 1-14 catalogs to get the spectroscopic parameters for each object

The DA physical parameters are in
- DR14 table3
- DR12 table6
- DR10 table6
- DR7 table2
- DR4 table1
- DR1 table3

Preference the DR14 parameters then the DR12 then the DR10 and so on so that the most recent catalog is the one from which I get these measurements. Add a flag to indicate which catalog these spectroscopic parameters come from.

### Kepler 2019 DR14
https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.2169K/abstract

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/486/2169

 Examining the spectra of all stars with 3$\sigma$ proper motion in the Sloan Digital Sky Survey Data Release 14, we report the classification for 20088 spectroscopically confirmed white dwarfs. We obtain Teff, logg, and mass for hydrogen atmospherewhite dwarf stars (DAs). In addition to searching all spectra with significant proper motion for new white dwarfs, we also fitted known DAs and DBs that fell in our selection criteria. The SDSS flux calibration is based on hundreds of comparison stars and in general more accurate than those derived from single night observations. We fitted the spectra of highest S/N for each star, taking into account that SDSS re-observes fields and improves the quality of the spectra. Our classifications are independent from previous classifications, and should be considered improvements.

Of the total 37053 objects in our Table 2, only 6 per cent come from plates obtained after DR12, but only 13927 are in the SDSS DR7 to DR12 catalogues. The DR7 to DR12 catalogues contain 35590 stars, including 29262 DAs, so our catalogues are not a subset or complete sets, but complementary. The total number of unique stars in Kleinman et al. (2013ApJS..204....5K, Cat. J/ApJS/204/5), Kepler et al. (2015MNRAS.446.4078K, Cat. J/MNRAS/446/4078, 2016MNRAS.455.3413K, Cat. J/MNRAS/455/3413), and this DR14 catalogue is 52299, with 28681 DAs.

table2.dat Spectral classification

table3.dat Parameters from spectral fitting to atmospheric models for DAs

table5.dat Table of hot subdwarfs

In [10]:
filename='keplerDR14_table3.dat.gz.txt'

#read in the DR14 table data
DR14_df = pd.read_csv('data/'+filename,delimiter='|')
#drop the last row of NaNs and the first row of dashes
DR14_df=DR14_df.dropna()
DR14_df=DR14_df.drop([0])
#reset the indices
indices=np.arange(0,len(DR14_df))
DR14_df.set_index(indices,inplace=True)
#remove space from column names
DR14_df.columns = DR14_df.columns.str.replace(' ', '')
#separate into plate, mjd, and fiber columns
plates=[]
mjds=[]
fibers=[]
for pmf in DR14_df['P-M-F'].to_list():
    plate,mjd,fiber=pmf.split('-')
    plates.append(plate)
    mjds.append(mjd)
    fibers.append(fiber)
DR14_df['plate']=plates
DR14_df['mjd']=mjds
DR14_df['fiber']=fibers
#standardize PMFs to compare across data releases
cols=['plate','mjd','fiber']
for col in cols:
    DR14_df[col]=DR14_df[col].astype(int) #convert first to integer to strip 0s
    DR14_df[col]=DR14_df[col].astype(str) #convert back to string
#add unique PMF identifyer
DR14_df['pmf']=DR14_df['plate']+DR14_df['mjd']+DR14_df['fiber']
DR14_df['pmf']=DR14_df['pmf'].astype(int)
#rename columns
DR14_df=DR14_df.rename(columns={'S/N':'snr_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff","logg":"logg1d","e_logg":"e_logg","RV":"rv",
                                "e_RV":"e_rv","Mass":"mass1d","e_Mass":"e_mass1d","Teff3D":"teff","logg3D":"logg",
                                "Mass3D":"mass","e_":"e_mass"})
#replace ' ---  ' signifying no measurement with np.nans
DR14_df=DR14_df.replace(' ---  ', np.nan)
DR14_df=DR14_df.replace('---  ', np.nan)
#convert to floats
cols=['snr_sdss_dr','teff', 'e_teff', 'logg', 'e_logg', 'rv', 'e_rv','mass', 'e_mass','teff1d','logg1d','mass1d','e_mass1d']
for col in cols:
    DR14_df[col]=DR14_df[col].astype(float)
#calculate the radius in solar units
DR14_df['radius']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*DR14_df['mass']/10**DR14_df['logg'])
DR14_df['radius1d']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*DR14_df['mass1d']/10**DR14_df['logg1d'])
#calculate errors on the radius
DR14_df['e_radius']=DR14_df['radius']*np.sqrt(((1/(4*(DR14_df['mass'])**2))*(DR14_df['e_mass'])**2)+((1/4)*((np.log(10))**2)*(DR14_df['e_logg'])**2))
DR14_df['e_radius1d']=DR14_df['radius1d']*np.sqrt(((1/(4*(DR14_df['mass1d'])**2))*(DR14_df['e_mass1d'])**2)+((1/4)*((np.log(10))**2)*(DR14_df['e_logg'])**2))
#convert the SDSS name to coordinates
ras=[]
decs=[]
for i in range(len(DR14_df)):
    hour=DR14_df['SDSSJ'][i][:2]
    minit=DR14_df['SDSSJ'][i][2:4]
    sec=DR14_df['SDSSJ'][i][4:9]
    sign=DR14_df['SDSSJ'][i][9:10]
    deg=DR14_df['SDSSJ'][i][10:12]
    arcmin=DR14_df['SDSSJ'][i][12:14]
    arcsec=DR14_df['SDSSJ'][i][14:]
    ra=str(hour)+'h'+str(minit)+'m'+str(sec)+'s'
    dec=str(sign)+str(deg)+'d'+str(arcmin)+'m'+str(arcsec)+'s'
    coord=SkyCoord(ra, dec)
    ras.append(float(coord.to_string('decimal').split()[0]))
    decs.append(float(coord.to_string('decimal').split()[1]))
DR14_df['ra_sdss_dr']=ras
DR14_df['dec_sdss_dr']=decs

#drop all extra columns
DR14_df=DR14_df.drop(['P-M-F', 'SDSSJ','SNg','umag', 'e_umag', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'imag', 'e_imag', 'zmag',
                  'e_zmag', 'E(B-V)', 'PM','GLONGLAT', 'DT(SP)', 'd', 'z','DTeff', 'Dlogg',
                  'plate', 'mjd', 'fiber','Type'],axis=1)


In [11]:
DR14_df

,teff1d,e_teff,logg1d,e_logg,rv,e_rv,snr_sdss_dr,mass1d,e_mass1d,teff,logg,mass,e_mass,pmf,radius,radius1d,e_radius,e_radius1d,ra_sdss_dr,dec_sdss_dr
0,11493.0,122.0,8.396,0.079,64.0,22.0,10.0,0.841,0.049,11437.0,8.235,0.73677,0.03484,26651602314,0.010849,0.009630,0.001020,0.000920,145.281,-0.330472
1,10809.0,27.0,8.094,0.025,-18.0,5.0,36.0,0.654,0.014,10702.0,7.875,0.53674,0.00847,2665163026,0.014015,0.012023,0.000418,0.000369,147.255,-0.319336
2,10135.0,50.0,8.525,0.068,25.0,14.0,14.0,0.920,0.042,10033.0,8.255,0.74933,0.03043,2665163031,0.010692,0.008682,0.000865,0.000708,147.018,-0.127267
3,11614.0,152.0,8.437,0.095,-28.0,23.0,10.0,0.867,0.059,11568.0,8.281,0.77233,0.04207,2665163037,0.010535,0.009327,0.001187,0.001068,147.321,-0.006575
4,19971.0,126.0,7.922,0.022,43.0,7.0,30.0,0.583,0.010,19971.0,7.922,0.58328,0.00723,26651630570,0.013841,0.013837,0.000361,0.000370,146.668,1.222180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15467,45237.0,1208.0,7.636,0.136,0.0,0.0,0.0,0.000,0.000,45237.0,7.636,0.00000,0.00000,887357427955,0.000000,0.000000,NaN,NaN,176.211,37.424400
15468,9027.0,18.0,8.014,0.022,-4.0,4.0,27.0,0.606,0.012,8987.0,7.744,0.48362,0.00697,887457426176,0.015469,0.012690,0.000407,0.000345,173.023,36.570800
15469,14749.0,119.0,8.035,0.023,34.0,8.0,24.0,0.630,0.012,14749.0,8.035,0.62970,0.00856,887457426466,0.012627,0.012630,0.000345,0.000355,171.119,36.713700
15470,10344.0,54.0,8.374,0.051,59.0,13.0,13.0,0.825,0.032,10236.0,8.104,0.65924,0.02062,887457426594,0.011933,0.009783,0.000725,0.000605,171.667,38.200200


### Kepler 2016 DR12
https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.3413K

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/455/3413

We report the discovery of 6576 new spectroscopically confirmed white dwarf and subdwarf stars in the Sloan Digital Sky Survey Data Release 12. We obtain Teff, logg and mass for hydrogen atmosphere white dwarf stars (DAs) and helium atmosphere white dwarf stars (DBs). We found 2936 new hydrogen-dominated white dwarf stars. We calculate the mass distribution of all 5883 DAs with S/N>=15 in DR12, including the ones in DR7 and DR10, with an average S/N=26, corrected to the 3D convection scale, and also the distribution after correcting for the observed volume, using 1/Vmax.

table6.dat New white dwarf stars

table7.dat New classification of known white dwarf stars

In [12]:
def read_DR12(filename):
    #read in the DR12 data
    DR12_df = pd.read_csv('data/'+filename,delimiter='|')
    #drop the last row of NaNs and the first row of dashes
    DR12_df=DR12_df.drop([len(DR12_df)-1])
    DR12_df=DR12_df.drop([0])
    #reset the indices
    indices=np.arange(0,len(DR12_df))
    DR12_df.set_index(indices,inplace=True)
    #remove space from column names and columns
    DR12_df.columns = DR12_df.columns.str.replace(' ', '')
    for col in DR12_df.columns.to_list():
        DR12_df[col]=DR12_df[col].str.replace(' ', '')
    #separate into plate, mjd, and fiber columns
    plates=[]
    mjds=[]
    fibers=[]
    for pmf in DR12_df['PMF'].to_list():
        plate,mjd,fiber=pmf.split('-')
        plates.append(plate)
        mjds.append(mjd)
        fibers.append(fiber)
    DR12_df['plate']=plates
    DR12_df['mjd']=mjds
    DR12_df['fiber']=fibers
    #standardize PMFs to compare across data releases
    cols=['plate','mjd','fiber']
    for col in cols:
        DR12_df[col]=DR12_df[col].astype(int) #convert first to integer to strip 0s
        DR12_df[col]=DR12_df[col].astype(str) #convert back to string
    #add unique PMF identifyer
    DR12_df['pmf']=DR12_df['plate']+DR12_df['mjd']+DR12_df['fiber']
    DR12_df['pmf']=DR12_df['pmf'].astype(int)
    #rename columns
    DR12_df=DR12_df.rename(columns={'S/N':'snr_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff1d","logg": "logg1d","e_logg": "e_logg1d", "e_RV":"e_rv",
                                "Mass":"mass","e_Mass":"e_mass","Teff_3D":"teff","e_Teff_3D":"e_teff",
                                "logg_3D":"logg","e_logg_3D":"e_logg"})
    #replace ' ---  ' signifying no measurement with np.nans
    DR12_df=DR12_df.replace('---', np.nan)
    #replace empty strings also signifying no measurements with np.nans
    DR12_df=DR12_df.replace(r'^\s*$', np.nan, regex=True)
    #convert to floats
    cols=['snr_sdss_dr','teff', 'e_teff', 'logg', 'e_logg', 'teff1d', 'e_teff1d', 'logg1d', 'e_logg1d', 'mass',
          'e_mass']
    for col in cols:
        DR12_df[col]=DR12_df[col].astype(float)
    #calculate the radius in solar units
    DR12_df['radius']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*DR12_df['mass']/10**DR12_df['logg'])
    #calculate errors on the radius
    DR12_df['e_radius']=DR12_df['radius']*np.sqrt(((1/(4*(DR12_df['mass'])**2))*(DR12_df['e_mass'])**2)+((1/4)*((np.log(10))**2)*(DR12_df['e_logg'])**2))
    #convert the SDSS name to coordinates
    ras=[]
    decs=[]
    for i in range(len(DR12_df)):
        hour=DR12_df['SDSS'][i][:2]
        minit=DR12_df['SDSS'][i][2:4]
        sec=DR12_df['SDSS'][i][4:9]
        sign=DR12_df['SDSS'][i][9:10]
        deg=DR12_df['SDSS'][i][10:12]
        arcmin=DR12_df['SDSS'][i][12:14]
        arcsec=DR12_df['SDSS'][i][14:]
        ra=str(hour)+'h'+str(minit)+'m'+str(sec)+'s'
        dec=str(sign)+str(deg)+'d'+str(arcmin)+'m'+str(arcsec)+'s'
        coord=SkyCoord(ra, dec)
        ras.append(float(coord.to_string('decimal').split()[0]))
        decs.append(float(coord.to_string('decimal').split()[1]))
    DR12_df['ra_sdss_dr']=ras
    DR12_df['dec_sdss_dr']=decs 
    #drop all extra columns
    DR12_df=DR12_df.drop(['PMF','SDSS','SpType','umag', 'e_umag', 'gmag', 'e_gmag', 'rmag','e_rmag', 'imag', 'e_imag', 'zmag', 'e_zmag',
                  'E(B-V)', 'pm','GLONGLAT','Com', 'plate','mjd', 'fiber'],axis=1)

    return(DR12_df)


In [13]:
#read in the DR12 table6 data
DR12_df_table6=read_DR12('keplerDR12_table6.dat.gz.txt')
#read in the DR12 table7 data
DR12_df_table7=read_DR12('keplerDR12_table7.dat.txt')

In [14]:
DR12_df_table6

,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,teff,e_teff,logg,e_logg,mass,e_mass,pmf,radius,e_radius,ra_sdss_dr,dec_sdss_dr
0,22.0,29108.0,463.0,5.409,0.087,NaN,NaN,NaN,NaN,NaN,NaN,33751997195,NaN,NaN,192.842,-2.62700
1,22.0,29876.0,374.0,5.292,0.080,NaN,NaN,NaN,NaN,NaN,NaN,6145343749,NaN,NaN,232.803,55.02770
2,46.0,30893.0,165.0,5.033,0.032,NaN,NaN,NaN,NaN,NaN,NaN,79052346634,NaN,NaN,221.738,58.15560
3,11.0,29404.0,704.0,5.450,0.142,NaN,NaN,NaN,NaN,NaN,NaN,105952618515,NaN,NaN,117.315,28.11140
4,45.0,30503.0,191.0,5.019,0.036,NaN,NaN,NaN,NaN,NaN,NaN,117752824636,NaN,NaN,326.616,-7.18822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6642,13.0,16016.0,137.0,8.025,0.041,16016.0,137.0,8.03,0.04,0.630,0.021,751656782855,0.012703,0.000622,137.198,46.20920
6643,3.0,12463.0,1883.0,8.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN,751756772524,NaN,NaN,137.096,46.35400
6644,20.0,12475.0,151.0,5.828,0.048,12483.0,151.0,5.83,0.05,0.177,0.050,751756772816,0.084764,0.012928,139.290,46.63940
6645,6.0,10395.0,299.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,751756772851,NaN,NaN,139.674,46.66920


In [15]:
DR12_df_table7

,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,teff,e_teff,logg,e_logg,mass,e_mass,pmf,radius,e_radius,ra_sdss_dr,dec_sdss_dr
0,22.0,7957.0,1000.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,716756604806,NaN,NaN,0.226667,-9.13526
1,64.0,28566.0,99.0,5.503,0.017,NaN,NaN,NaN,NaN,NaN,NaN,687956539704,NaN,NaN,0.779417,24.20320
2,7.0,8264.0,97.0,7.157,0.218,8259.0,97.0,6.94,0.22,0.23400,0.03900,714756574956,0.027154,0.007240,0.788583,-6.04264
3,8.0,8489.0,99.0,8.164,0.162,8500.0,99.0,7.93,0.16,0.55900,0.07600,714856591586,0.013425,0.002636,0.788583,-6.04264
4,15.0,6126.0,103.0,9.320,0.214,6110.0,103.0,9.37,0.21,1.28506,0.07184,436555539502,0.003879,0.000944,0.840000,-1.88635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,12.0,7713.0,68.0,7.731,0.112,7733.0,68.0,7.58,0.11,0.40789,0.04420,43585583326,0.017159,0.002364,353.887000,-2.46121
405,29.0,46000.0,3000.0,7.000,0.250,NaN,NaN,NaN,NaN,NaN,NaN,616156238392,NaN,NaN,354.330000,6.41779
406,35.0,12378.0,224.0,8.868,0.168,NaN,NaN,NaN,NaN,1.10400,0.08500,616156238622,NaN,NaN,354.575000,8.62568
407,46.0,42517.0,313.0,7.877,0.026,NaN,NaN,NaN,NaN,0.57500,0.01100,61375627013,NaN,NaN,354.806000,15.47740


### Kepler 2015 DR10
https://ui.adsabs.harvard.edu/abs/2015MNRAS.446.4078K/abstract

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/446/4078

We report the discovery of 9088 new spectroscopically confirmed white dwarfs and subdwarfs in the Sloan Digital Sky Survey Data Release 10. We obtain T_eff_, logg and mass for hydrogen atmosphere white dwarf stars (DAs) and helium atmosphere white dwarf stars (DBs). We found 6887 new hydrogen-dominated white dwarf stars.

table6.dat New white dwarf stars

In [16]:
#read in the DR10 data
DR10_df = pd.read_csv('data/keplerDR10_table6.dat.gz.txt',delimiter='|')
#drop the last row of NaNs and the first row of dashes
DR10_df=DR10_df.drop([len(DR10_df)-1])
DR10_df=DR10_df.drop([0])
#reset the indices
indices=np.arange(0,len(DR10_df))
DR10_df.set_index(indices,inplace=True)
#remove space from column names and type column
DR10_df.columns = DR10_df.columns.str.replace(' ', '')
#separate into plate, mjd, and fiber columns
plates=[]
mjds=[]
fibers=[]
for pmf in DR10_df['PMJ'].to_list():
    plate,mjd,fiber=pmf.split('-')
    plates.append(plate)
    mjds.append(mjd)
    fibers.append(fiber)
DR10_df['plate']=plates
DR10_df['mjd']=mjds
DR10_df['fiber']=fibers
#standardize PMFs to compare across data releases
cols=['plate','mjd','fiber']
for col in cols:
    DR10_df[col]=DR10_df[col].astype(int) #convert first to integer to strip 0s
    DR10_df[col]=DR10_df[col].astype(str) #convert back to string
#add unique PMF identifyer
DR10_df['pmf']=DR10_df['plate']+DR10_df['mjd']+DR10_df['fiber']
DR10_df['pmf']=DR10_df['pmf'].astype(int)
#rename columns
DR10_df=DR10_df.rename(columns={'S/N':'snr_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff1d","logg": "logg1d","e_logg": "e_logg1d", "e_RV":"e_rv",
                            "Mass":"mass","e_Mass":"e_mass","Teff_3D":"teff","e_Teff_3D":"e_teff",
                            "logg_3D":"logg","e_logg_3D":"e_logg"})
#replace ' ---  ' signifying no measurement with np.nans
DR10_df=DR10_df.replace('---', np.nan)
#replace empty strings also signifying no measurements with np.nans
DR10_df=DR10_df.replace(r'^\s*$', np.nan, regex=True)
#convert to floats
cols=['snr_sdss_dr','teff', 'e_teff', 'logg', 'e_logg', 'teff1d', 'e_teff1d', 'logg1d', 'e_logg1d', 'mass',
      'e_mass']
for col in cols:
    DR10_df[col]=DR10_df[col].astype(float)
#calculate the radius in solar units
DR10_df['radius']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*DR10_df['mass']/10**DR10_df['logg'])
#calculate errors on the radius
DR10_df['e_radius']=DR10_df['radius']*np.sqrt(((1/(4*(DR10_df['mass'])**2))*(DR10_df['e_mass'])**2)+((1/4)*((np.log(10))**2)*(DR10_df['e_logg'])**2))
#convert the SDSS name to coordinates
ras=[]
decs=[]
for i in range(len(DR10_df)):
    hour=DR10_df['SDSS'][i][:2]
    minit=DR10_df['SDSS'][i][2:4]
    sec=DR10_df['SDSS'][i][4:9]
    sign=DR10_df['SDSS'][i][9:10]
    deg=DR10_df['SDSS'][i][10:12]
    arcmin=DR10_df['SDSS'][i][12:14]
    arcsec=DR10_df['SDSS'][i][14:]
    ra=str(hour)+'h'+str(minit)+'m'+str(sec)+'s'
    dec=str(sign)+str(deg)+'d'+str(arcmin)+'m'+str(arcsec)+'s'
    coord=SkyCoord(ra, dec)
    ras.append(float(coord.to_string('decimal').split()[0]))
    decs.append(float(coord.to_string('decimal').split()[1]))
DR10_df['ra_sdss_dr']=ras
DR10_df['dec_sdss_dr']=decs 
#drop all extra columns
DR10_df=DR10_df.drop(['SDSS','Type','PMJ','umag', 'e_umag', 'gmag', 'e_gmag', 'rmag','e_rmag', 'imag', 'e_imag', 'zmag',
                  'e_zmag', 'pm','Com', 'plate', 'mjd', 'fiber'],axis=1)

In [17]:
DR10_df

,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,teff,e_teff,logg,e_logg,mass,e_mass,pmf,radius,e_radius,ra_sdss_dr,dec_sdss_dr
0,19.0,11323.0,124.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,421655477816,NaN,NaN,0.298583,0.061819
1,26.0,11107.0,90.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,421655477200,NaN,NaN,0.318708,0.034569
2,2.0,10464.0,433.0,8.584,0.212,10422.0,433.0,8.31,0.21,0.787,0.130,564955912467,0.010285,0.002628,0.566792,12.052600
3,45.0,20153.0,80.0,8.178,0.001,20153.0,80.0,8.18,0.00,0.716,0.000,453555860842,0.011394,0.000000,0.679833,7.648960
4,13.0,6302.0,6.0,5.430,0.240,NaN,NaN,NaN,NaN,NaN,NaN,453455863584,NaN,NaN,0.696708,10.195600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9107,10.0,36903.0,1000.0,7.849,0.187,NaN,NaN,NaN,NaN,0.550,0.091,282454452364,NaN,NaN,359.471000,28.761900
9108,3.0,8134.0,283.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,427755506748,NaN,NaN,359.544000,4.754960
9109,20.0,7300.0,1000.0,6.500,0.500,NaN,NaN,NaN,NaN,NaN,NaN,441555831544,NaN,NaN,359.829000,6.355310
9110,10.0,7537.0,62.0,8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,427755506860,NaN,NaN,359.859000,3.799600


### Kleinman 2013 DR7
https://ui.adsabs.harvard.edu/abs/2013ApJS..204....5K

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/204/5

We present a new catalog of spectroscopically confirmed white dwarf stars from the Sloan Digital Sky Survey (SDSS) Data Release 7 spectroscopic catalog. We find 20407 white dwarf spectra, representing 19712 stars, and provide atmospheric model fits to 14120 DA and 1011 DB white dwarf spectra from 12843 and 923 stars, respectively. 

Our catalog includes all white dwarf stars from the earlier Kleinman et al. (2004, Cat. J/ApJ/607/426) and Eisenstein et al. (2006, Cat. J/ApJS/167/40) catalogs, although occasionally with different identifications.

table2.dat Catalog of white dwarfs

In [18]:
#read in the DR7 data
DR7_astpy = Table.read('data/kleinmanDR7_table2.dat.fits.gz')
#convert to pandas
DR7_df=DR7_astpy.to_pandas()
#rename columns
DR7_df=DR7_df.rename(columns={"Plate":"plate","Fiber":"fiber","MJD":"mjd"})
#standardize PMFs to compare across data releases
cols=['plate','mjd','fiber']
for col in cols:
    DR7_df[col]=DR7_df[col].astype(int) #convert first to integer to strip 0s
    DR7_df[col]=DR7_df[col].astype(str) #convert back to string
#add unique PMF identifyer
DR7_df['pmf']=DR7_df['plate']+DR7_df['mjd']+DR7_df['fiber']
DR7_df['pmf']=DR7_df['pmf'].astype(int)
#rename columns
DR7_df=DR7_df.rename(columns={'SNg':'snr_sdss_dr','RAdeg':'ra_sdss_dr','DEdeg':'dec_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff1d","log(g)": "logg1d","e_log(g)": "e_logg1d",
                              "Mass":"mass1d","e_Mass":"e_mass1d"})
#calculate the radius in solar units
DR7_df['radius1d']=(1/RsunCGS)*np.sqrt(GCGS*MsunCGS*DR7_df['mass1d']/10**DR7_df['logg1d'])
#calculate errors on the radius
DR7_df['e_radius1d']=DR7_df['radius1d']*np.sqrt(((1/(4*(DR7_df['mass1d'])**2))*(DR7_df['e_mass1d'])**2)+((1/4)*((np.log(10))**2)*(DR7_df['e_logg1d'])**2))
#drop all extra columns
DR7_df=DR7_df.drop(['SDSS','Type','plate', 'mjd', 'fiber', 'umag','e_umag', 'f_umag', 'gmag', 'e_gmag', 'f_gmag', 'rmag',
                  'e_rmag', 'f_rmag', 'imag', 'e_imag', 'f_imag', 'zmag', 'e_zmag', 'f_zmag', 'pm', 'pmPA',
                  'f_pm', 'Ag', 'GMT', 'Atype','chi2', 'm_Nsp', 'Nsp', 'q_Nsp'],axis=1)



In [19]:
DR7_df

,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,mass1d,e_mass1d,pmf,radius1d,e_radius1d
0,0.028139,-0.781661,12.34160,10772,111,8.37,0.099,0.824,0.064,68552203225,0.009822,0.001183
1,0.029825,-9.727734,7.46310,29354,701,7.55,0.143,0.477,0.053,65052143497,0.019208,0.003338
2,0.048197,-8.835659,10.36020,10112,47,9.23,0.575,0.000,0.000,65052143450,0.000000,NaN
3,0.051353,-0.845158,4.55945,7994,164,8.33,0.311,0.794,0.194,109152902117,0.010096,0.003819
4,0.093897,-10.861716,11.56350,8598,72,8.32,0.128,0.789,0.084,65052143217,0.010180,0.001595
...,...,...,...,...,...,...,...,...,...,...,...,...
20402,359.954360,24.100752,14.02230,21734,515,7.87,0.073,0.563,0.032,282254389281,0.014437,0.001281
20403,359.956640,25.690153,10.18390,20200,510,7.67,0.087,0.484,0.041,282254389340,0.016852,0.001833
20404,359.961200,1.127802,12.19040,10424,86,8.25,0.093,0.748,0.058,68552203425,0.010744,0.001223
20405,359.973070,28.028305,17.65630,25506,426,7.84,0.057,0.559,0.027,282454452255,0.014891,0.001041


### Eisenstein 2006 DR4
https://ui.adsabs.harvard.edu/abs/2006ApJS..167...40E/abstract

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/167/40

We present a catalog of 9316 spectroscopically confirmed white dwarfs from the Sloan Digital Sky Survey Data Release 4.

table1.dat SDSS DR4 white dwarfs

table1c.dat SDSS DR4 hot Subdwarfs

table5.dat List of stars from DR1 catalog missing from this catalog (skip this since we will just use the DR1 catalog)


In [20]:
#edit the DR4 table1 .txt file to separate SpClass and WD columns and RA/Dec columns
"""
file = open('SDSS_data/eisensteinDR4_table1.dat.gz.txt', "r")
lines=file.readlines()
newlines=[]
for line in lines:
    list_line=list(line)
    list_line[11]='|'
    list_line[67]='|'
    newlines.append("".join(list_line))
newfile = open('SDSS_data/eisensteinDR4_table1_2.dat.gz.txt', "a")
newfile.writelines(newlines)
newfile.close()
"""
#we don't have to edit the other files since they don't contain DAs

'\nfile = open(\'SDSS_data/eisensteinDR4_table1.dat.gz.txt\', "r")\nlines=file.readlines()\nnewlines=[]\nfor line in lines:\n    list_line=list(line)\n    list_line[11]=\'|\'\n    list_line[67]=\'|\'\n    newlines.append("".join(list_line))\nnewfile = open(\'SDSS_data/eisensteinDR4_table1_2.dat.gz.txt\', "a")\nnewfile.writelines(newlines)\nnewfile.close()\n'

In [21]:
filename='eisensteinDR4_table1_2.dat.gz.txt'
#read in the DR4 data
DR4_df = pd.read_csv('data/'+filename,delimiter='|')
#drop the last row of NaNs and the first row of dashes
DR4_df=DR4_df.drop([len(DR4_df)-1])
DR4_df=DR4_df.drop([0])
#reset the indices
indices=np.arange(0,len(DR4_df))
DR4_df.set_index(indices,inplace=True)
#remove space from column names
DR4_df.columns = DR4_df.columns.str.replace(' ', '')
#rename columns
DR4_df=DR4_df.rename(columns={"MJD-S":"mjd","Plate":"plate","Fiber":"fiber"})
#standardize PMFs to compare across data releases
cols=['plate','mjd','fiber']
for col in cols:
    DR4_df[col]=DR4_df[col].astype(int) #convert first to integer to strip 0s
    DR4_df[col]=DR4_df[col].astype(str) #convert back to string
#add unique PMF identifyer
DR4_df['pmf']=DR4_df['plate']+DR4_df['mjd']+DR4_df['fiber']
DR4_df['pmf']=DR4_df['pmf'].astype(int)
#rename columns
DR4_df=DR4_df.rename(columns={'RAdeg':'ra_sdss_dr','DEdeg':'dec_sdss_dr','S/N':'snr_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff1d","log(g)": "logg1d","e_log(g)": "e_logg1d"})
#convert to floats
cols=['ra_sdss_dr','dec_sdss_dr','snr_sdss_dr','teff1d', 'e_teff1d', 'logg1d', 'e_logg1d']
for col in cols:
    DR4_df[col]=DR4_df[col].astype(float)
#drop all extra columns
DR4_df=DR4_df.drop(['WD', 'SpClass','SDSS','r_SpClass','pm', 'pmPA','pmRA', 'pmDE', 'MJD-P', 'Run', 'reRun', 'CC',
                  'Field', 'ID', 'umag','gmag', 'rmag', 'imag', 'zmag', 'e_umag', 'e_gmag', 'e_rmag', 'e_imag',
                  'e_zmag', 'f_umag', 'f_gmag', 'f_rmag', 'f_imag', 'f_zmag', 'Ag','plate', 'fiber',
                  'mjd','Deblend', 'D-PTfl', 'H-PTfl', 'D-STfl','H-STfl', 'QFlag', 'AElem','chi2'],axis=1)

    

In [22]:
DR4_df

,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,pmf
0,0.028147,-0.781616,12.76,10881.0,108.0,8.32,0.086,68552203225
1,0.029836,-9.727753,7.75,28669.0,652.0,7.52,0.136,65052143497
2,0.048182,-8.835651,11.06,10106.0,43.0,8.14,0.624,65052143450
3,0.051346,-0.845157,4.53,8397.0,151.0,8.72,0.225,109152902117
4,0.093902,-10.861710,12.18,8793.0,64.0,8.59,0.131,65052143217
...,...,...,...,...,...,...,...,...
10085,359.222057,0.360766,9.59,7519.0,75.0,8.81,0.164,68452523596
10086,359.607505,-10.570337,35.12,22450.0,180.0,7.93,0.026,65052143233
10087,359.674622,-1.004806,6.51,42353.0,3203.0,6.93,0.480,68552203222
10088,359.905534,0.325716,8.24,10207.0,116.0,8.19,0.137,68552203451


### Kleinman 2004 DR1
https://ui.adsabs.harvard.edu/abs/2004ApJ...607..426K

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJ/607/426

We present the full spectroscopic white dwarf and hot subdwarf sample from the Sloan Digital Sky Survey (SDSS) First Data Release, DR1. We find 2551 white dwarf stars of various types. Of the white dwarf stars, 1888 are nonmagnetic DA types and 171 are nonmagnetic DBs. We fit the DA and DB spectra with a grid of models to determine the T_eff_ and logg for each object. 

table3.dat SDSS DR1 spectroscopically identified DA stars

table4.dat SDSS DR1 spectroscopically identified DB stars

table5.dat All SDSS DR1 spectroscopically identified white dwarf and subdwarf stars
                           
table6.dat All SDSS DR1 spectroscopically identified white dwarf and subdwarf stars with uncertain identifications
                           
table11.dat Previously known white dwarf stars recovered in this work
                           
table12.dat Previously known white dwarf stars not spectroscopically recovered in this work

There are no earlier classifications to check against DR1, so we only need table3 for DR1

In [23]:
#read in the DR1 data
DR1_astpy = Table.read('data/kleinmanDR1_table3.dat.fits')
#convert to pandas
DR1_df=DR1_astpy.to_pandas()
#rename columns
DR1_df=DR1_df.rename(columns={"ObsDate":"mjd","Plate":"plate","Fiber":"fiber"})
#standardize PMFs to compare across data releases
cols=['plate','mjd','fiber']
for col in cols:
    DR1_df[col]=DR1_df[col].astype(int) #convert first to integer to strip 0s
    DR1_df[col]=DR1_df[col].astype(str) #convert back to string
#add unique PMF identifyer
DR1_df['pmf']=DR1_df['plate']+DR1_df['mjd']+DR1_df['fiber']
DR1_df['pmf']=DR1_df['pmf'].astype(int)
#rename columns
DR1_df=DR1_df.rename(columns={'RAdeg':'ra_sdss_dr','DEdeg':'dec_sdss_dr','S/N':'snr_sdss_dr',"Teff":"teff1d","e_Teff": "e_teff1d","log(g)": "logg1d","e_log(g)": "e_logg1d"})
#drop all extra columns
DR1_df=DR1_df.drop(['---','SDSS','plate', 'mjd', 'fiber','f_umag','umag', 'e_umag', 'f_gmag', 'gmag', 'e_gmag', 'f_rmag',
                  'rmag','e_rmag', 'f_imag', 'imag', 'e_imag', 'f_zmag', 'zmag', 'e_zmag','pmRA', 'pmDE',
                  'Ag','Chi2','AutoID', 'Notes'],axis=1)

In [24]:
DR1_df

,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff1d,e_teff1d,logg1d,e_logg1d,pmf
0,1.34173,-9.54185,3.61,0,0,0.00,0.00,65152141382
1,2.67918,-0.89987,5.78,0,0,0.00,0.00,388517931
2,3.41294,0.32358,70.39,0,0,0.00,0.00,38951795431
3,9.95403,14.70684,10.61,0,0,0.00,0.00,41951879318
4,10.21404,14.72644,6.88,0,0,0.00,0.00,41951879280
...,...,...,...,...,...,...,...,...
1883,13.94907,-8.75202,46.28,81159,2091,7.39,0.08,65752177569
1884,321.04997,-7.44688,71.23,83582,1247,7.72,0.05,64052178273
1885,262.45991,58.30243,11.32,90975,6897,7.71,0.39,36652017151
1886,188.65494,66.72683,15.09,91304,6285,7.37,0.26,4945191552


### Crossmatch and add to the dataframe

In [25]:
RAs=[]
Decs=[]
SNRs=[]
Teffs=[]
E_Teffs=[]
Loggs=[]
E_Loggs=[]
Teff_1Ds=[]
E_Teff_1Ds=[]
Logg_1Ds=[]
E_Logg_1Ds=[]
RVs=[]
E_RVs=[]
Masss=[]
E_Masss=[]
Mass_1Ds=[]
E_Mass_1Ds=[]
Radiuss=[]
E_Radiuss=[]
Radiuss_1Ds=[]
E_Radiuss_1Ds=[]
SDSS_DRs=[]
for i in range(len(eSDSS_DA_df)):
    tmp=eSDSS_DA_df.loc[i]
    pmf=tmp['pmf']
    crossmatch=DR14_df.query('pmf==@pmf')
    if len(crossmatch)>0:
        RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
        Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
        SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
        Teffs.append(crossmatch['teff'].to_list()[0])
        E_Teffs.append(crossmatch['e_teff'].to_list()[0])
        Loggs.append(crossmatch['logg'].to_list()[0])
        E_Loggs.append(crossmatch['e_logg'].to_list()[0])
        Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
        E_Teff_1Ds.append(crossmatch['e_teff'].to_list()[0]) 
        Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
        E_Logg_1Ds.append(crossmatch['e_logg'].to_list()[0])
        RVs.append(crossmatch['rv'].to_list()[0])
        E_RVs.append(crossmatch['e_rv'].to_list()[0])
        Masss.append(crossmatch['mass'].to_list()[0])
        E_Masss.append(crossmatch['e_mass'].to_list()[0])
        Mass_1Ds.append(crossmatch['mass1d'].to_list()[0])
        E_Mass_1Ds.append(crossmatch['e_mass1d'].to_list()[0])
        Radiuss.append(crossmatch['radius'].to_list()[0])
        E_Radiuss.append(crossmatch['e_radius'].to_list()[0])
        Radiuss_1Ds.append(crossmatch['radius1d'].to_list()[0])
        E_Radiuss_1Ds.append(crossmatch['e_radius1d'].to_list()[0])
        SDSS_DRs.append('dr14')
    else:
        crossmatch=DR12_df_table6.query('pmf==@pmf')
        if len(crossmatch)>0:
            RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
            Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
            SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
            Teffs.append(crossmatch['teff'].to_list()[0])
            E_Teffs.append(crossmatch['e_teff'].to_list()[0])
            Loggs.append(crossmatch['logg'].to_list()[0])
            E_Loggs.append(crossmatch['e_logg'].to_list()[0])
            Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
            E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
            Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
            E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
            RVs.append(np.nan)
            E_RVs.append(np.nan)
            Masss.append(crossmatch['mass'].to_list()[0])
            E_Masss.append(crossmatch['e_mass'].to_list()[0])
            Mass_1Ds.append(np.nan)
            E_Mass_1Ds.append(np.nan)
            Radiuss.append(crossmatch['radius'].to_list()[0])
            E_Radiuss.append(crossmatch['e_radius'].to_list()[0])
            Radiuss_1Ds.append(np.nan)
            E_Radiuss_1Ds.append(np.nan)
            SDSS_DRs.append('dr12_table6')
        else:
            crossmatch=DR12_df_table7.query('pmf==@pmf')
            if len(crossmatch)>0:
                RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
                Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
                SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
                Teffs.append(crossmatch['teff'].to_list()[0])
                E_Teffs.append(crossmatch['e_teff'].to_list()[0])
                Loggs.append(crossmatch['logg'].to_list()[0])
                E_Loggs.append(crossmatch['e_logg'].to_list()[0])
                Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
                E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
                Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
                E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
                RVs.append(np.nan)
                E_RVs.append(np.nan)
                Masss.append(crossmatch['mass'].to_list()[0])
                E_Masss.append(crossmatch['e_mass'].to_list()[0])
                Mass_1Ds.append(np.nan)
                E_Mass_1Ds.append(np.nan)
                Radiuss.append(crossmatch['radius'].to_list()[0])
                E_Radiuss.append(crossmatch['e_radius'].to_list()[0])
                Radiuss_1Ds.append(np.nan)
                E_Radiuss_1Ds.append(np.nan)
                SDSS_DRs.append('dr12_table7')
            else:
                crossmatch=DR10_df.query('pmf==@pmf')
                if len(crossmatch)>0:
                    RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
                    Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
                    SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
                    Teffs.append(crossmatch['teff'].to_list()[0])
                    E_Teffs.append(crossmatch['e_teff'].to_list()[0])
                    Loggs.append(crossmatch['logg'].to_list()[0])
                    E_Loggs.append(crossmatch['e_logg'].to_list()[0])
                    Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
                    E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
                    Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
                    E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
                    RVs.append(np.nan)
                    E_RVs.append(np.nan)
                    Masss.append(crossmatch['mass'].to_list()[0])
                    E_Masss.append(crossmatch['e_mass'].to_list()[0])
                    Mass_1Ds.append(np.nan)
                    E_Mass_1Ds.append(np.nan)
                    Radiuss.append(crossmatch['radius'].to_list()[0])
                    E_Radiuss.append(crossmatch['e_radius'].to_list()[0])
                    Radiuss_1Ds.append(np.nan)
                    E_Radiuss_1Ds.append(np.nan)
                    SDSS_DRs.append('dr10')
                else:
                    crossmatch=DR7_df.query('pmf==@pmf')
                    if len(crossmatch)>0:
                        RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
                        Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
                        SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
                        Teffs.append(np.nan)
                        E_Teffs.append(np.nan)
                        Loggs.append(np.nan)
                        E_Loggs.append(np.nan)
                        Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
                        E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
                        Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
                        E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
                        RVs.append(np.nan)
                        E_RVs.append(np.nan)
                        Masss.append(np.nan)
                        E_Masss.append(np.nan)
                        Mass_1Ds.append(crossmatch['mass1d'].to_list()[0])
                        E_Mass_1Ds.append(crossmatch['e_mass1d'].to_list()[0])
                        Radiuss.append(np.nan)
                        E_Radiuss.append(np.nan)
                        Radiuss_1Ds.append(crossmatch['radius1d'].to_list()[0])
                        E_Radiuss_1Ds.append(crossmatch['e_radius1d'].to_list()[0])
                        SDSS_DRs.append('dr7')
                    else:
                        crossmatch=DR4_df.query('pmf==@pmf')
                        if len(crossmatch)>0:
                            RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
                            Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
                            SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
                            Teffs.append(np.nan)
                            E_Teffs.append(np.nan)
                            Loggs.append(np.nan)
                            E_Loggs.append(np.nan)
                            Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
                            E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
                            Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
                            E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
                            RVs.append(np.nan)
                            E_RVs.append(np.nan)
                            Masss.append(np.nan)
                            E_Masss.append(np.nan)
                            Mass_1Ds.append(np.nan)
                            E_Mass_1Ds.append(np.nan)
                            Radiuss.append(np.nan)
                            E_Radiuss.append(np.nan)
                            Radiuss_1Ds.append(np.nan)
                            E_Radiuss_1Ds.append(np.nan)
                            SDSS_DRs.append('dr4')
                        else:
                            crossmatch=DR1_df.query('pmf==@pmf')
                            if len(crossmatch)>0:
                                RAs.append(crossmatch['ra_sdss_dr'].to_list()[0])
                                Decs.append(crossmatch['dec_sdss_dr'].to_list()[0])
                                SNRs.append(crossmatch['snr_sdss_dr'].to_list()[0])
                                Teffs.append(np.nan)
                                E_Teffs.append(np.nan)
                                Loggs.append(np.nan)
                                E_Loggs.append(np.nan)
                                Teff_1Ds.append(crossmatch['teff1d'].to_list()[0])
                                E_Teff_1Ds.append(crossmatch['e_teff1d'].to_list()[0])
                                Logg_1Ds.append(crossmatch['logg1d'].to_list()[0])
                                E_Logg_1Ds.append(crossmatch['e_logg1d'].to_list()[0])
                                RVs.append(np.nan)
                                E_RVs.append(np.nan)
                                Masss.append(np.nan)
                                E_Masss.append(np.nan)
                                Mass_1Ds.append(np.nan)
                                E_Mass_1Ds.append(np.nan)
                                Radiuss.append(np.nan)
                                E_Radiuss.append(np.nan)
                                Radiuss_1Ds.append(np.nan)
                                E_Radiuss_1Ds.append(np.nan)
                                SDSS_DRs.append('dr1')
                            else:
                                RAs.append(np.nan)
                                Decs.append(np.nan)
                                SNRs.append(np.nan)
                                Teffs.append(np.nan)
                                E_Teffs.append(np.nan)
                                Loggs.append(np.nan)
                                E_Loggs.append(np.nan)
                                Teff_1Ds.append(np.nan)
                                E_Teff_1Ds.append(np.nan)
                                Logg_1Ds.append(np.nan)
                                E_Logg_1Ds.append(np.nan)
                                RVs.append(np.nan)
                                E_RVs.append(np.nan)
                                Masss.append(np.nan)
                                E_Masss.append(np.nan)
                                Mass_1Ds.append(np.nan)
                                E_Mass_1Ds.append(np.nan)
                                Radiuss.append(np.nan)
                                E_Radiuss.append(np.nan)
                                Radiuss_1Ds.append(np.nan)
                                E_Radiuss_1Ds.append(np.nan)
                                SDSS_DRs.append('none')
    

In [26]:
eSDSS_DA_df['sdss_dr']=SDSS_DRs
eSDSS_DA_df['ra_sdss_dr']=RAs
eSDSS_DA_df['dec_sdss_dr']=Decs
eSDSS_DA_df['snr_sdss_dr']=SNRs
eSDSS_DA_df['teff_sdss_dr']=Teffs
eSDSS_DA_df['e_teff_sdss_dr']=E_Teffs
eSDSS_DA_df['logg_sdss_dr']=Loggs
eSDSS_DA_df['e_logg_sdss_dr']=E_Loggs
eSDSS_DA_df['teff1d_sdss_dr']=Teff_1Ds
eSDSS_DA_df['e_teff1d_sdss_dr']=E_Teff_1Ds
eSDSS_DA_df['logg1d_sdss_dr']=Logg_1Ds
eSDSS_DA_df['e_logg1d_sdss_dr']=E_Logg_1Ds
eSDSS_DA_df['rv_sdss_dr']=RVs
eSDSS_DA_df['e_rv_sdss_dr']=E_RVs
eSDSS_DA_df['mass_sdss_dr']=Masss
eSDSS_DA_df['e_mass_sdss_dr']=E_Masss
eSDSS_DA_df['mass1d_sdss_dr']=Mass_1Ds
eSDSS_DA_df['e_mass1d_sdss_dr']=E_Mass_1Ds
eSDSS_DA_df['radius_sdss_dr']=Radiuss
eSDSS_DA_df['e_radius_sdss_dr']=E_Radiuss
eSDSS_DA_df['radius1d_sdss_dr']=Radiuss_1Ds
eSDSS_DA_df['e_radius1d_sdss_dr']=E_Radiuss_1Ds

In [27]:
#save as .csv
eSDSS_DA_df.to_csv('csv/eSDSS_DA_df.csv',index=False)

In [28]:
eSDSS_DA_df

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr
0,781593995293031680,4689,55656,251,468955656251,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,781593995293031680,1362,53050,530,136253050530,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,54.0,26946.0,84.0,7.978,0.013,26946.0,84.0,7.978,0.013,25.0,4.0,0.62583,0.00440,0.626,0.006,0.013442,0.000207,0.013443,0.000211
2,781593995293031680,8359,57449,142,835957449142,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,48.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,45.0,26502.0,98.0,7.928,0.015,26502.0,98.0,7.928,0.015,33.0,5.0,0.60092,0.00495,0.601,0.007,0.013952,0.000248,0.013953,0.000254
3,3091672976133787776,1184,52641,347,118452641347,123.145644,4.147560,13.866804,0.103250,-64.653153,0.079957,2739.666081,3.154774,17.093122,2181.492391,14.250237,16.991657,917.540894,8.798259,17.341331,1.131172,218.627921,19.921849,31.0,25120.46,1376.69,7.820046,0.105608,0.542125,0.050888,0.015005,0.001956,False,dr14,123.14600,4.14781,30.0,27228.0,146.0,7.805,0.023,27228.0,146.0,7.805,0.023,10.0,8.0,0.54696,0.00668,0.547,0.009,0.015336,0.000417,0.015336,0.000425
4,923453157308619904,3670,55480,564,367055480564,115.728835,40.059892,-55.909583,0.112149,-24.052275,0.070936,1675.078079,1.949819,17.627279,1216.617095,13.342116,17.625658,654.720232,14.962528,17.707756,1.117164,179.368400,26.435002,11.0,13230.80,920.87,7.831787,0.109121,0.515890,0.056472,0.014441,0.001979,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25410,3785343312929168256,2876,54581,362,287654581362,171.841816,-6.392256,-8.892839,0.582126,-1.387960,0.402545,200.852821,0.981598,19.930172,165.236145,8.486315,19.793280,60.404485,8.335972,20.295223,1.123413,268.610795,50.791014,12.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,False,dr14,171.84200,-6.39226,10.0,31672.0,436.0,7.620,0.102,31672.0,436.0,7.620,0.102,-28.0,35.0,0.52124,0.01556,0.521,0.017,0.018524,0.002193,0.018520,0.002196
25411,3878309048281561472,1236,52751,404,123652751404,151.096303,9.160865,-15.685595,0.418477,-10.498901,0.337166,397.739508,1.394981,19.188370,281.971361,8.186472,19.213030,155.790500,10.230367,19.266542,1.100625,229.376816,46.716852,8.0,12748.65,2382.41,8.344622,0.307349,0.822779,0.194159,0.010105,0.003769,False,dr14,151.09600,9.16090,7.0,11107.0,172.0,7.739,0.146,111

# Get the Bailer-Jones Distances from Gaia

https://www2.mpia-hd.mpg.de/~calj/gedr3_distances.html
- The Bailer-Jones source id is the same as in EDR3
- r med geo, r lo geo, and r hi geo are the median, 16th percentile, and 84th percentile of the geometric distance posterior in parsec


In [29]:
def get_gaia_info(wd_table):
    stardats = []
    id_list=tuple(wd_table['gaia_dr3_source_id'].unique())
    iters = (len(id_list)+100) // 100

    #query gaia to get the photometry and Bailer-Jones distances for each object
    for i in tqdm(range(iters)):
        ADQL_CODE1 = """SELECT dist.source_id, dist.r_lo_geo, dist.r_med_geo, dist.r_hi_geo 
        FROM gaiadr3.gaia_source as gaia
        JOIN external.gaiaedr3_distance as dist
        ON gaia.source_id = dist.source_id      
        WHERE gaia.source_id in {}""".format(id_list[100*i:100*i+100])
        stardats.append(Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
    
    gaia_astpy = vstack(stardats)
    gaia_astpy.rename_column('source_id', 'gaia_dr3_source_id')
    gaia_df=gaia_astpy.to_pandas()

    catalog=pd.merge(left=wd_table,right=gaia_df,on='gaia_dr3_source_id',how="inner")
    
    #check to make sure distances exist for each object
    nodist=np.isnan(catalog['r_med_geo'].to_list())
    catalog['no_distance']=nodist
    return(catalog)

In [30]:
eSDSS_DA_df=get_gaia_info(eSDSS_DA_df)

100%|█████████████████████████████████████████| 193/193 [02:09<00:00,  1.49it/s]


# Get the SDSS Photometry

### The DR data sets contain the SDSS magnitudes but these are on the weird nanomaggie unit scale and are from earlier iterations of the SDSS photometric reductions. So, we want to get the latest SDSS photometry to be consistent across all DRs.

The SDSS imaging camera was retired in 2009. So, there is no new SDSS photometry available, only new reductions of old photometric data.

- https://en.wikipedia.org/wiki/Sloan_Digital_Sky_Survey
- https://www.sdss.org/instruments/

Use astroquery to get the photometry given the coordinates of each object.

A full list of all photoobj_fields that can be queried, along with a description of each field, can be found here
https://github.com/astropy/astroquery/blob/main/astroquery/sdss/data/PhotoObjAll_dr12.json
- Download the table and read in the data

Here is a description of the differrent SDSS measures of flux and magnitude:
https://live-sdss4org-dr12.pantheonsite.io/algorithms/magnitudes/
- Isolated stars are best described by PSF magnitudes
- Galactic extinction corrections in magnitudes at the position of each object, extinction, are computed following Schlegel, Finkbeiner & Davis (1998). These corrections are not applied to the magnitudes in the databases unless otherwise noted; if desired, they must be explicitly applied by the user. The conversions we use from E(B–V) to total extinction Aλ, assuming a z=0 elliptical galaxy spectral energy distribution, are tabulated in Table 22 of the EDR Paper.

The columns we care about are:
- clean: Clean photometry flag (1=clean, 0=unclean) (see https://www.sdss4.org/dr17/tutorials/flags/)
- probPSF: Probability that the object is a star. Currently 0 if type == 3 (galaxy), 1 if type == 6 (star).
- flags: Photo Object Attribute Flags
- psfMag_PASSBAND: PASSBAND={u,g,r,i,z}, PSF magnitude
- psfMagErr_PASSBAND: PASSBAND={u,g,r,i,z}, PSF magnitude error
- psfFlux_PASSBAND: PASSBAND={u,g,r,i,z}, PSF flux in nanomaggies
- psfFluxIvar_PASSBAND: PASSBAND={u,g,r,i,z}, PSF flux inverse variance in nanomaggies^{-2}
- ra: J2000 Right Ascension (r-band)
- dec: J2000 Declination (r-band)
- extinction_PASSBAND: PASSBAND={u,g,r,i,z}, Extinction in passband





In [31]:
def get_sdss_info(wd_table):
    #get set of coordinates
    sdss_phot_df=wd_table[['gaia_dr3_source_id','ra','dec']]
    coords = SkyCoord(sdss_phot_df['ra'], sdss_phot_df['dec'], unit='deg')

    stardats = []
    phot_inds=[]

    #query sdss to get the photometry for each object
    photoobj_fields_list=['clean','probPSF',
             'psfMag_u','psfMag_g','psfMag_r','psfMag_i','psfMag_z',
             'psfMagErr_u','psfMagErr_g','psfMagErr_r','psfMagErr_i','psfMagErr_z',
             'psfFlux_u','psfFlux_g','psfFlux_r','psfFlux_i','psfFlux_z',
             'psfFluxIvar_u','psfFluxIvar_g','psfFluxIvar_r','psfFluxIvar_i','psfFluxIvar_z']
    clean=[]
    probPSF=[]
    psfMag_u=[]
    psfMag_g=[]
    psfMag_r=[]
    psfMag_i=[]
    psfMag_z=[]
    psfMagErr_u=[]
    psfMagErr_g=[]
    psfMagErr_r=[]
    psfMagErr_i=[]
    psfMagErr_z=[]
    psfFlux_u=[]
    psfFlux_g=[]
    psfFlux_r=[]
    psfFlux_i=[]
    psfFlux_z=[]
    psfFluxIvar_u=[]
    psfFluxIvar_g=[]
    psfFluxIvar_r=[]
    psfFluxIvar_i=[]
    psfFluxIvar_z=[]
    for coord in tqdm(coords):
        try:
            SDSS_query = SDSS.query_crossid(coord, photoobj_fields=photoobj_fields_list)
            clean.append(SDSS_query['clean'][0])
            probPSF.append(SDSS_query['probPSF'][0])
            psfMag_u.append(SDSS_query['psfMag_u'][0])
            psfMag_g.append(SDSS_query['psfMag_g'][0])
            psfMag_r.append(SDSS_query['psfMag_r'][0])
            psfMag_i.append(SDSS_query['psfMag_i'][0])
            psfMag_z.append(SDSS_query['psfMag_z'][0])
            psfMagErr_u.append(SDSS_query['psfMagErr_u'][0])
            psfMagErr_g.append(SDSS_query['psfMagErr_g'][0])
            psfMagErr_r.append(SDSS_query['psfMagErr_r'][0])
            psfMagErr_i.append(SDSS_query['psfMagErr_i'][0])
            psfMagErr_z.append(SDSS_query['psfMagErr_z'][0])
            psfFlux_u.append(SDSS_query['psfFlux_u'][0])
            psfFlux_g.append(SDSS_query['psfFlux_g'][0])
            psfFlux_r.append(SDSS_query['psfFlux_r'][0])
            psfFlux_i.append(SDSS_query['psfFlux_i'][0])
            psfFlux_z.append(SDSS_query['psfFlux_z'][0])
            psfFluxIvar_u.append(SDSS_query['psfFluxIvar_u'][0])
            psfFluxIvar_g.append(SDSS_query['psfFluxIvar_g'][0])
            psfFluxIvar_r.append(SDSS_query['psfFluxIvar_r'][0])
            psfFluxIvar_i.append(SDSS_query['psfFluxIvar_i'][0])
            psfFluxIvar_z.append(SDSS_query['psfFluxIvar_z'][0])
        except:
            #if no SDSS photometry, fill with np.nan
            clean.append(np.nan)
            probPSF.append(np.nan)
            psfMag_u.append(np.nan)
            psfMag_g.append(np.nan)
            psfMag_r.append(np.nan)
            psfMag_i.append(np.nan)
            psfMag_z.append(np.nan)
            psfMagErr_u.append(np.nan)
            psfMagErr_g.append(np.nan)
            psfMagErr_r.append(np.nan)
            psfMagErr_i.append(np.nan)
            psfMagErr_z.append(np.nan)
            psfFlux_u.append(np.nan)
            psfFlux_g.append(np.nan)
            psfFlux_r.append(np.nan)
            psfFlux_i.append(np.nan)
            psfFlux_z.append(np.nan)
            psfFluxIvar_u.append(np.nan)
            psfFluxIvar_g.append(np.nan)
            psfFluxIvar_r.append(np.nan)
            psfFluxIvar_i.append(np.nan)
            psfFluxIvar_z.append(np.nan)
    sdss_phot_df['clean']=clean
    sdss_phot_df['prob_psf']=probPSF
    sdss_phot_df['psf_mag_u']=psfMag_u
    sdss_phot_df['psf_mag_g']=psfMag_g
    sdss_phot_df['psf_mag_r']=psfMag_r
    sdss_phot_df['psf_mag_i']=psfMag_i
    sdss_phot_df['psf_mag_z']=psfMag_z
    sdss_phot_df['psf_magerr_u']=psfMagErr_u
    sdss_phot_df['psf_magerr_g']=psfMagErr_g
    sdss_phot_df['psf_magerr_r']=psfMagErr_r
    sdss_phot_df['psf_magerr_i']=psfMagErr_i
    sdss_phot_df['psf_magerr_z']=psfMagErr_z
    sdss_phot_df['psf_flux_u']=psfFlux_u
    sdss_phot_df['psf_flux_g']=psfFlux_g
    sdss_phot_df['psf_flux_r']=psfFlux_r
    sdss_phot_df['psf_flux_i']=psfFlux_i
    sdss_phot_df['psf_flux_z']=psfFlux_z
    sdss_phot_df['psf_fluxivar_u']=psfFluxIvar_u
    sdss_phot_df['psf_fluxivar_g']=psfFluxIvar_g
    sdss_phot_df['psf_fluxivar_r']=psfFluxIvar_r
    sdss_phot_df['psf_fluxivar_i']=psfFluxIvar_i
    sdss_phot_df['psf_fluxivar_z']=psfFluxIvar_z

    sdss_phot_df=sdss_phot_df.drop(['ra','dec','gaia_dr3_source_id'],axis=1)

    #merge on index since for each entry have 1 set of SDSS phot (or np.nans)
    catalog=pd.merge(left=wd_table,right=sdss_phot_df,left_index=True,right_index=True,how="inner")

    return(catalog)

In [32]:
eSDSS_DA_df=get_sdss_info(eSDSS_DA_df)

  0%|                                                 | 0/25415 [00:00<?, ?it/s]/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|████████████████████████████████████| 25415/25415 [00:29<00:00, 867.04it/s]


### Convert SDSS magnitudes to the AB system

The SDSS flux is in weird units (nanomaggies) and the magnitude system is nearly the AB system but not quite and is also in asinh magnitudes instead of normal magnitudes. (see Conversion from SDSS ugriz magnitudes to AB ugriz magnitudes at https://live-sdss4org-dr12.pantheonsite.io/algorithms/fluxcal/#SDSStoAB, also see http://faraday.uwyo.edu/~admyers/ASTR5160/handouts/516016.pdf). So first we must convert the fluxes to normal magnitudes on the SDSS system
$$m=-2.5\text{log}_{10}(f/f_0)$$
where the zero-point is $3631\times10^9\text{ Jy}=10^9 f_0$, plugging into the expression for $m$, if $f$ is in nanomaggies then the magnitude is
$$m=22.5-2.5\text{log}_{10}(f)$$

Then apply the corrections to the AB system from https://iopscience.iop.org/article/10.1086/507110
$$u_{AB}=u_{SDSS}-0.040\text{ mag}$$
$$g_{AB}=g_{SDSS}\text{ mag}$$
$$r_{AB}=r_{SDSS}\text{ mag}$$
$$i_{AB}=i_{SDSS}+0.015\text{ mag}$$
$$z_{AB}=z_{SDSS}+0.030\text{ mag}$$

These statements are not precise to better than 0.01 mag, so add that error in quadrature.

In [33]:
eSDSS_DA_df['mag_ab_u']=22.5-2.5*np.log10(eSDSS_DA_df['psf_flux_u'])-0.04
eSDSS_DA_df['magerr_ab_u']=np.sqrt(eSDSS_DA_df['psf_magerr_u']**2+0.01**2)

eSDSS_DA_df['mag_ab_g']=22.5-2.5*np.log10(eSDSS_DA_df['psf_flux_g'])
eSDSS_DA_df['magerr_ab_g']=np.sqrt(eSDSS_DA_df['psf_magerr_g']**2+0.01**2)

eSDSS_DA_df['mag_ab_r']=22.5-2.5*np.log10(eSDSS_DA_df['psf_flux_r'])
eSDSS_DA_df['magerr_ab_r']=np.sqrt(eSDSS_DA_df['psf_magerr_r']**2+0.01**2)

eSDSS_DA_df['mag_ab_i']=22.5-2.5*np.log10(eSDSS_DA_df['psf_flux_i'])+0.015
eSDSS_DA_df['magerr_ab_i']=np.sqrt(eSDSS_DA_df['psf_magerr_i']**2+0.01**2)

eSDSS_DA_df['mag_ab_z']=22.5-2.5*np.log10(eSDSS_DA_df['psf_flux_z'])+0.03
eSDSS_DA_df['magerr_ab_z']=np.sqrt(eSDSS_DA_df['psf_magerr_z']**2+0.01**2)

/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  res

In [34]:
#for photometric fitting, we will use SDSS u+r+z bands 
#flag any object that is missing SDSS photometry or has negative fluxes
nophotmag_u=np.isnan(eSDSS_DA_df['mag_ab_u'].to_list())
nophotmag_r=np.isnan(eSDSS_DA_df['mag_ab_r'].to_list())
nophotmag_z=np.isnan(eSDSS_DA_df['mag_ab_z'].to_list())
nophotmagerr_u=np.isnan(eSDSS_DA_df['magerr_ab_u'].to_list())
nophotmagerr_r=np.isnan(eSDSS_DA_df['magerr_ab_r'].to_list())
nophotmagerr_z=np.isnan(eSDSS_DA_df['magerr_ab_z'].to_list())

#flag any object with 0 SDSS flux (infinite magnitude)
zero_u=np.isnan(eSDSS_DA_df['mag_ab_u'].where(eSDSS_DA_df['mag_ab_u']<np.inf).to_list())
zero_r=np.isnan(eSDSS_DA_df['mag_ab_r'].where(eSDSS_DA_df['mag_ab_r']<np.inf).to_list())
zero_z=np.isnan(eSDSS_DA_df['mag_ab_z'].where(eSDSS_DA_df['mag_ab_z']<np.inf).to_list())

nosdssphot=[]
for i in range(len(nophotmag_u)):
    if nophotmag_u[i]==False and nophotmag_r[i]==False and nophotmag_z[i]==False and nophotmagerr_u[i]==False and nophotmagerr_r[i]==False and nophotmagerr_z[i]==False and zero_u[i]==False and zero_r[i]==False and zero_z[i]==False:
        nosdssphot.append(False)
    else:
        nosdssphot.append(True)
eSDSS_DA_df['no_sdss_phot']=nosdssphot


In [35]:
display(eSDSS_DA_df.query('no_sdss_phot==True'))

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot
305,666722243106163968,2080,53350,111,208053350111,119.246287,15.692126,3.692555,0.146326,-3.571068,0.087362,1126.255210,1.563696,18.058275,855.318666,9.267995,18.008223,403.863478,9.874000,18.232309,1.118026,205.720619,21.410959,23.0,17608.19,1752.12,8.018080,0.167058,0.628772,0.096274,0.012866,0.002663,False,dr7,119.24628,15.692145,22.153999,NaN,NaN,NaN,NaN,17949.0,179.0,7.990,0.035,NaN,NaN,NaN,NaN,0.613,0.021,NaN,NaN,0.013121,0.000574,241.460266,247.817383,254.404373,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
306,666722243106163968,4494,55569,450,449455569450,119.246287,15.692126,3.692555,0.146326,-3.571068,0.087362,1126.255210,1.563696,18.058275,855.318666,9.267995,18.008223,403.863478,9.874000,18.232309,1.118026,205.720619,21.410959,35.0,17608.19,1752.12,8.018080,0.167058,0.628772,0.096274,0.012866,0.002663,False,dr14,119.24600,15.692100,0.000000,17929.0,109.0,7.998,0.022,17929.0,109.0,7.998,0.022,0.0,0.0,0.61683,0.00784,0.000,0.000,0.013041,0.000341,0.000000,NaN,241.460266,247.817383,254.404373,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
444,732128788225576832,2870,54534,161,287054534161,165.438782,28.277042,-30.194458,0.632268,3.189856,0.652743,122.374233,0.820791,20.468142,79.561807,7.774767,20.586781,58.074147,11.432975,20.337938,1.124714,203.803743,65.746956,7.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,False,dr7,165.43887,28.277029,6.512990,NaN,NaN,NaN,NaN,10876.0,211.0,8.200,0.191,NaN,NaN,NaN,NaN,0.718,0.121,NaN,NaN,0.011150,0.002626,582.590393,907.013428,1390.935303,False,0.0,0.0,20.96558,20.93346,24.80207,24.36162,22.79863,0.064620,0.054133,3.012934,1.787045,0.531156,4.104522,4.230756,-0.000006,0.000066,0.038566,16.679180,22.43394,2.254483,2.848213,1.906279,20.926844,0.065389,20.933955,0.055049,NaN,3.012951,32.972133,1.787073,26.064475,0.531250,True
659,1595902060698469376,1165,52703,635,116552703635,230.055857,51.926625,10.812482,0.936938,0.110512,1.280461,101.918254,0.825270,20.666737,76.891238,9.926597,20.623850,59.974122,9.085437,20.302986,1.342893,84.900340,52.763070,4.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435.633484,613.025024,921.833252,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
891,581069149132840064,3811,55538,340,381155538340,131.763443,3.209091,-1.462939,0.215523,-1.670116,0.150163,621.835649,0.971573,18.703178,457.297107,7.534068,18.688046,217.973464,6.428489,18.901886,1.085931,223.952746,27.066275,27.0,17258.26,2015.06,8.682549,0.153713,1.040074,0.088

# Save the table of earlier SDSS DA WDs as a .csv file and import back in if needed


In [36]:
#save result
eSDSS_DA_df.to_csv('csv/eSDSS_DA_df.csv',index=False)

In [37]:
#import back in the data
eSDSS_DA_df=pd.read_csv('csv/eSDSS_DA_df.csv')
display(eSDSS_DA_df)

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot
0,781593995293031680,4689,55656,251,468955656251,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False
1,781593995293031680,1362,53050,530,136253050530,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,54.0,26946.0,84.0,7.978,0.013,26946.0,84.0,7.978,0.013,25.0,4.0,0.62583,0.00440,0.626,0.006,0.013442,0.000207,0.013443,0.000211,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False
2,781593995293031680,8359,57449,142,835957449142,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,48.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,45.0,26502.0,98.0,7.928,0.015,26502.0,98.0,7.928,0.015,33.0,5.0,0.60092,0.00495,0.601,0.007,0.013952,0.000248,0.013953,0.000254,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False
3,3091672976133787776,1184,52641,347,118452641347,123.145644,4.147560,13.866804,0.103250,-64.653153,0.079957,2739.666081,3.154774,17.093122,2181.492391,14.250236,16.991657,917.540894,8.798259,17.341331,1.131172,218.627921,19.921849,31.0,25120.46,1376.69,7.820046,0.105608,0.542125,0.050888,0.015005,0.001956,False,dr14,123.14600,4.14781,30.0,27228.0,146.0,7.805,0.023,27228.0,146.0,7.805,0.023,10.0,8.0,0.54696,0.00668,0.547,0.009,0.015336,0.000417,0.015336,0.000425,244.32657,250.09187,257.23975,False,1.0,1

In [38]:
len(eSDSS_DA_df['gaia_dr3_source_id'].unique())

19257

# Get all spectra


In [39]:
#break into sets of ~500 observations
num_tables=int(len(eSDSS_DA_df)/500)+1
print(num_tables)

51


In [40]:
match_tab_dict={}
for i in tqdm(range(num_tables)):
    match_tab_dict['set_'+str(i)]=eSDSS_DA_df[500*i:500*i+500]
    #reset the indices
    indices=np.arange(0,len(match_tab_dict['set_'+str(i)]))
    match_tab_dict['set_'+str(i)].set_index(indices,inplace=True)

100%|█████████████████████████████████████████| 51/51 [00:00<00:00, 6134.31it/s]


In [41]:
def parallelprocess_get_spectra(i):
    #get SDSS spectra from DR18 Science Archive Server (SAS)
    
    urls=['https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/26/spectra/lite/',
          'https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/103/spectra/lite/',
          'https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/104/spectra/lite/',
          'https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/v5_13_2/spectra/lite/',
          'https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/v6_0_4/spectra/lite/',
         'https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/eFEDS/spectra/lite/',
         'https://dr18.sdss.org/sas/dr18/spectro/boss/redux/eFEDS/spectra/lite/',
         'https://dr18.sdss.org/sas/dr18/spectro/boss/redux/v6_0_4/spectra/lite/']
    
    snrs=[]
    spec_flags=[]
    for j in range(len(match_tab_dict['set_'+str(i)])):
        tmp=match_tab_dict['set_'+str(i)].loc[j]
        plate=str(tmp['plate'])
        if len(plate)<4:
            plate='0'+plate
        mjd=str(tmp['mjd'])
        fiber=str(tmp['fiber'])
        if len(fiber)==1:
            fiber='000'+fiber
        if len(fiber)==2:
            fiber='00'+fiber
        if len(fiber)==3:
            fiber='0'+fiber
        filename=str(tmp['pmf'])
        fitsfilename='spec-'+plate+'-'+mjd+'-'+fiber+'.fits'

        for k in range(len(urls)):
            base_url=urls[k]
            found_spec=False
            try:
                url=base_url+plate+'/'+fitsfilename 
                with fits.open(url) as spec:
                    # Get pertinent spectrum data    
                    wavl = np.array(10**spec[1].data['loglam'])
                    flux = np.array(spec[1].data['flux'])
                    ivar = np.array(spec[1].data['ivar'])
                    snr=np.array(spec[2].data['SN_MEDIAN_ALL'][0])
                #write the data in a txt file
                np.savez('csv/eSDSS_spectra/'+filename,flux=flux,wavl=wavl,ivar=ivar,snr=snr)
                snrs.append(snr)
                spec_flags.append(k)
                found_spec=True
                break
            except:
                continue
        
        if found_spec==False:
            snrs.append(np.nan)
            spec_flags.append(np.nan)
            
    match_tab_dict['set_'+str(i)]['snr']=snrs
    match_tab_dict['set_'+str(i)]['spec_avail']=spec_flags
    #save the short set to the backup, we won't use these again
    match_tab_dict['set_'+str(i)].to_csv('csv/get_spec_short_sets/'+str(i)+'.csv',index=False)
    

In [42]:
#get spectra
print(num_tables)
#set n_jobs=-2 to spawn n-1 CPU threads where n=total number CPUs available
result=Parallel(n_jobs=-2, verbose=11)(delayed(parallelprocess_get_spectra)(i) for i in range(num_tables))

51


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-2)]: Done   2 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-2)]: Done   3 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-2)]: Done   5 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-2)]: Done   6 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-2)]: Done   7 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-2)]: Done   8 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-2)]: Done   9 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-2)]: Done  10 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-2)]: Done  12 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-2)]: Done  13 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-2)]: Done  14 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-2)]: Done  15 tasks      | elapsed: 12

In [43]:
#read back in those temporary tables
match_tab_dict={}
for i in tqdm(range(num_tables)):
    match_tab_dict['set_'+str(i)]=pd.read_csv('csv/get_spec_short_sets/'+str(i)+'.csv')
    

100%|██████████████████████████████████████████| 51/51 [00:00<00:00, 215.76it/s]


In [44]:
#combine all of the dfs into one df
match_tab=pd.concat([match_tab_dict['set_0'], match_tab_dict['set_1']], ignore_index=True)
for i in range(num_tables-2):
    match_tab=pd.concat([match_tab, match_tab_dict['set_'+str(i+2)]], ignore_index=True)
#save the results of searching for spectra
match_tab.to_csv('csv/eSDSS_DA_df.csv',index=False)
display(match_tab)

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot,snr,spec_avail
0,781593995293031680,4689,55656,251,468955656251,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False,47.991780,3.0
1,781593995293031680,1362,53050,530,136253050530,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,55.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,54.0,26946.0,84.0,7.978,0.013,26946.0,84.0,7.978,0.013,25.0,4.0,0.62583,0.00440,0.626,0.006,0.013442,0.000207,0.013443,0.000211,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False,51.459084,0.0
2,781593995293031680,8359,57449,142,835957449142,164.360613,43.351927,-52.019284,0.127783,-16.841075,0.152640,4403.739422,5.975801,16.577812,3802.343448,9.860086,16.388414,1657.477904,6.589113,16.699276,1.239815,170.284986,61.855434,48.0,22647.96,708.52,7.686154,0.065657,0.501285,0.018064,0.016834,0.001308,False,dr14,164.36100,43.35200,45.0,26502.0,98.0,7.928,0.015,26502.0,98.0,7.928,0.015,33.0,5.0,0.60092,0.00495,0.601,0.007,0.013952,0.000248,0.013953,0.000254,192.00090,197.97336,205.55202,False,1.0,1.0,16.17543,16.29479,16.69078,16.98537,17.28317,0.011920,0.017762,0.020509,0.018596,0.018326,338.70970,303.44850,210.711600,160.639300,122.100000,0.072321,0.040580,0.063124,0.132107,0.235413,16.135431,0.015559,16.294788,0.020383,16.690779,0.022817,17.000370,0.021114,17.313211,0.020877,False,42.292060,3.0
3,3091672976133787776,1184,52641,347,118452641347,123.145644,4.147560,13.866804,0.103250,-64.653153,0.079957,2739.666081,3.154774,17.093122,2181.492391,14.250236,16.991657,917.540894,8.798259,17.341331,1.131172,218.627921,19.921849,31.0,25120.46,1376.69,7.820046,0.105608,0.542125,0.050888,0.015005,0.001956,False,dr14,123.14600,4.14781,30.0,27228.0,146.0,7.805,0.023,27228.0,146.0,7.805,0.023,10.0,8.0,0.54696,0.00668,0.547,0.009,0.015336,0.000417,0.

In [45]:
#check for objects without spectra
nan_mask=match_tab['snr'].isnull()
nan_tab=match_tab[nan_mask]
#reset the indices
indices=np.arange(0,len(nan_tab))
nan_tab.set_index(indices,inplace=True)
display(nan_tab)

#the directory containing this plate is
#https://dr18.sdss.org/sas/dr18/spectro/sdss/redux/v5_13_2/spectra/lite/
#but the folder is empty

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot,snr,spec_avail
0,704174422349052288,10658,58439,194,1065858439194,128.463483,28.144191,-22.113177,0.273561,-27.811462,0.177919,452.076901,0.903871,19.049335,336.216755,6.128374,19.021994,154.411146,5.995458,19.276200,1.085275,195.738611,33.656767,17.0,18360.37,2832.77,8.391581,0.267973,0.861380,0.167385,0.009796,0.003168,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.90366,312.98508,336.81503,False,1.0,1.0,19.04461,18.87771,19.20435,19.46805,19.70433,0.022562,0.019855,0.019895,0.022356,0.075816,24.107010,28.113160,20.808700,16.320250,13.088430,3.984209,3.783414,6.877492,8.850883,1.182051,19.004642,0.024679,18.877726,0.022231,19.204388,0.022266,19.483183,0.024491,19.737781,0.076472,False,NaN,NaN
1,704690058942511616,10658,58439,989,1065858439989,129.548104,29.108268,-17.972178,0.620608,-8.949994,0.591424,154.364154,0.708862,20.216002,110.945808,6.237557,20.225765,61.966887,7.048532,20.267496,1.120161,194.907270,34.824458,7.0,13408.95,5177.54,8.210876,0.692814,0.737824,0.374968,0.011163,0.009344,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.05610,529.51670,700.86127,False,1.0,1.0,20.68511,20.18846,20.19455,20.40634,20.87715,0.058818,0.023576,0.030502,0.048754,0.214859,5.316864,8.405542,8.357751,6.873351,4.335182,12.020230,30.004400,18.124310,10.468770,1.216885,20.645861,0.059662,20.188586,0.025609,20.194776,0.032099,20.422079,0.049769,20.937482,0.215091,False,NaN,NaN
2,704116904146548864,10658,58439,294,1065858439294,128.008885,27.516915,24.302725,0.163466,-31.106089,0.124936,852.628589,1.285048,18.360468,638.292506,7.951157,18.325993,337.670460,9.518275,18.426662,1.144652,196.332165,33.100506,22.0,14341.36,1362.17,7.773231,0.182607,0.489578,0.081447,0.015049,0.003403,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266.28750,279.43204,295.26297,False,1.0,1.0,18.54370,18.23436,18.50000,18.76475,18.99142,0.024002,0.017844,0.017110,0.020102,0.042009,38.239610,50.845610,39.810270,31.194920,25.296540,1.399234,1.432083,2.540655,2.997428,1.040301,18.503716,0.026002,18.234366,0.020455,18.500012,0.019818,18.779790,0.022452,19.022347,0.043183,False,NaN,NaN
3,707646886227680768,10658,58439,694,1065858439694,127.747536,28.876202,-4.824206,0.280293,-6.865800,0.198062,425.143760,0.979628,19.116028,306.901912,10.134104,19.121044,151.941494,9.506967,19.293705,1.079266,194.693819,33.242396,18.0,15629.22,3998.10,7.399664,0.488732,0.380204,0.155635,0.020389,0.012208,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517.11957,594.51184,675.02480,False,1.0,1.0,18.89975,18.90754,19.29377,19.61797,20.15058,0.023005,0.020529,0.017653,0.022477,0.126978,27.547970,27.351380,19.163570,14.214870,8.642106,2.934824,3.738653,10.299

In [46]:
#get these spectra using astroquery
snrs=[]
for j in tqdm(range(len(nan_tab))):
    tmp=nan_tab.loc[j]
    plate=int(tmp['plate'])
    mjd=int(tmp['mjd'])
    fiber=int(tmp['fiber'])
    filename=str(tmp['pmf'])

    #access the spectrum 
    spec = SDSS.get_spectra_async(plate=plate, fiberID=fiber, mjd=mjd, data_release=16)[0].get_fits()
    #access the coadded spectrum data
    flux=np.array(spec[1].data['flux']) #flux in 10^-17 ergs/s/cm^2/angstrom
    wavl=np.array(10**spec[1].data['loglam']) #log of wavelength in angstroms, wavelength=10**spec[i].data['loglam']
    ivar=np.array(spec[1].data['ivar']) #inverse variance of the flux, the error on the flux is ivar^-0.5
    snr=np.array(spec[2].data['SN_MEDIAN_ALL'][0]) #median SNR from the spAll file
    #write the data in a txt file
    np.savez('csv/eSDSS_spectra/'+filename,flux=flux,wavl=wavl,ivar=ivar,snr=snr)
    snrs.append(snr)
nan_tab['snr']=snrs

  0%|                                                     | 0/5 [00:00<?, ?it/s]/Users/nicolecrumpler/anaconda3/envs/wd_models/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 19.88it/s]


In [47]:
#newly found spectra
nan_mask2=nan_tab['snr'].isnull()
found_spec=nan_tab[~nan_mask2]
#reset the indices
indices=np.arange(0,len(found_spec))
found_spec.set_index(indices,inplace=True)
found_spec

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot,snr,spec_avail
0,704174422349052288,10658,58439,194,1065858439194,128.463483,28.144191,-22.113177,0.273561,-27.811462,0.177919,452.076901,0.903871,19.049335,336.216755,6.128374,19.021994,154.411146,5.995458,19.276200,1.085275,195.738611,33.656767,17.0,18360.37,2832.77,8.391581,0.267973,0.861380,0.167385,0.009796,0.003168,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.90366,312.98508,336.81503,False,1.0,1.0,19.04461,18.87771,19.20435,19.46805,19.70433,0.022562,0.019855,0.019895,0.022356,0.075816,24.107010,28.113160,20.808700,16.320250,13.088430,3.984209,3.783414,6.877492,8.850883,1.182051,19.004642,0.024679,18.877726,0.022231,19.204388,0.022266,19.483183,0.024491,19.737781,0.076472,False,11.673271,NaN
1,704690058942511616,10658,58439,989,1065858439989,129.548104,29.108268,-17.972178,0.620608,-8.949994,0.591424,154.364154,0.708862,20.216002,110.945808,6.237557,20.225765,61.966887,7.048532,20.267496,1.120161,194.907270,34.824458,7.0,13408.95,5177.54,8.210876,0.692814,0.737824,0.374968,0.011163,0.009344,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.05610,529.51670,700.86127,False,1.0,1.0,20.68511,20.18846,20.19455,20.40634,20.87715,0.058818,0.023576,0.030502,0.048754,0.214859,5.316864,8.405542,8.357751,6.873351,4.335182,12.020230,30.004400,18.124310,10.468770,1.216885,20.645861,0.059662,20.188586,0.025609,20.194776,0.032099,20.422079,0.049769,20.937482,0.215091,False,4.483388,NaN
2,704116904146548864,10658,58439,294,1065858439294,128.008885,27.516915,24.302725,0.163466,-31.106089,0.124936,852.628589,1.285048,18.360468,638.292506,7.951157,18.325993,337.670460,9.518275,18.426662,1.144652,196.332165,33.100506,22.0,14341.36,1362.17,7.773231,0.182607,0.489578,0.081447,0.015049,0.003403,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266.28750,279.43204,295.26297,False,1.0,1.0,18.54370,18.23436,18.50000,18.76475,18.99142,0.024002,0.017844,0.017110,0.020102,0.042009,38.239610,50.845610,39.810270,31.194920,25.296540,1.399234,1.432083,2.540655,2.997428,1.040301,18.503716,0.026002,18.234366,0.020455,18.500012,0.019818,18.779790,0.022452,19.022347,0.043183,False,14.177101,NaN
3,707646886227680768,10658,58439,694,1065858439694,127.747536,28.876202,-4.824206,0.280293,-6.865800,0.198062,425.143760,0.979628,19.116028,306.901912,10.134104,19.121044,151.941494,9.506967,19.293705,1.079266,194.693819,33.242396,18.0,15629.22,3998.10,7.399664,0.488732,0.380204,0.155635,0.020389,0.012208,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517.11957,594.51184,675.02480,False,1.0,1.0,18.89975,18.90754,19.29377,19.61797,20.15058,0.023005,0.020529,0.017653,0.022477,0.126978,27.547970,27.351380,19.163570,14.214870,8.642106,2.93482

In [48]:
#add back these spectra
nan_mask2=nan_tab['snr'].isnull()
found_spec=nan_tab[~nan_mask2]
#reset the indices
indices=np.arange(0,len(found_spec))
found_spec.set_index(indices,inplace=True)
for i in range(len(found_spec)):
    tmp=found_spec.loc[i]
    plate=tmp['plate']
    mjd=tmp['mjd']
    fiber=tmp['fiber']
    ind=match_tab.query('plate==@plate & mjd==@mjd & fiber==@fiber').index[0]
    match_tab['snr'].loc[ind]=tmp['snr']
    match_tab['spec_avail'].loc[ind]='8'
    #display(match_tab.loc[ind:ind])
print(len(match_tab[match_tab['snr'].isnull()]))
print(len(match_tab[~match_tab['snr'].isnull()]))
#save the results of searching for spectra
match_tab.to_csv('csv/eSDSS_DA_df.csv',index=False)

0
25415


# Final Data Set

Remove the objects with no clean SDSS/Gaia photometry (there are none)


In [49]:
display(match_tab.query('(no_sdss_phot==True or clean==0) and (no_gaia_phot==True or phot_bp_rp_excess_factor>2)'))
#remove these objects
bad_ind=match_tab.query('(no_sdss_phot==True or clean==0) and (no_gaia_phot==True or phot_bp_rp_excess_factor>2)').index
match_tab=match_tab.drop(bad_ind)
#reset the indices
indices=np.arange(0,len(match_tab))
match_tab.set_index(indices,inplace=True)


,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot,snr,spec_avail


In [50]:
#sort by gaia dr3 source id
match_tab=match_tab.sort_values(by=['gaia_dr3_source_id'])
#reset the indices
indices=np.arange(0,len(match_tab))
match_tab.set_index(indices,inplace=True)

In [51]:
#save the results of searching for spectra
match_tab.to_csv('csv/eSDSS_DA_df.csv',index=False)

In [52]:
#save to backup directory
match_tab.to_csv('csv/eSDSS_DA_df_nb02.csv',index=False)

In [53]:
#import back in the data, either from the backup file or from the latest version of the table
backup=False
if backup==True:
    match_tab=pd.read_csv('csv/eSDSS_DA_df_nb02.csv')
else:
    match_tab=pd.read_csv('csv/eSDSS_DA_df.csv')
display(match_tab)

,gaia_dr3_source_id,plate,mjd,fiber,pmf,ra,dec,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,l,b,snr_gentile,teff_gentile,e_teff_gentile,logg_gentile,e_logg_gentile,mass_gentile,e_mass_gentile,radius_gentile,e_radius_gentile,no_gaia_phot,sdss_dr,ra_sdss_dr,dec_sdss_dr,snr_sdss_dr,teff_sdss_dr,e_teff_sdss_dr,logg_sdss_dr,e_logg_sdss_dr,teff1d_sdss_dr,e_teff1d_sdss_dr,logg1d_sdss_dr,e_logg1d_sdss_dr,rv_sdss_dr,e_rv_sdss_dr,mass_sdss_dr,e_mass_sdss_dr,mass1d_sdss_dr,e_mass1d_sdss_dr,radius_sdss_dr,e_radius_sdss_dr,radius1d_sdss_dr,e_radius1d_sdss_dr,r_lo_geo,r_med_geo,r_hi_geo,no_distance,clean,prob_psf,psf_mag_u,psf_mag_g,psf_mag_r,psf_mag_i,psf_mag_z,psf_magerr_u,psf_magerr_g,psf_magerr_r,psf_magerr_i,psf_magerr_z,psf_flux_u,psf_flux_g,psf_flux_r,psf_flux_i,psf_flux_z,psf_fluxivar_u,psf_fluxivar_g,psf_fluxivar_r,psf_fluxivar_i,psf_fluxivar_z,mag_ab_u,magerr_ab_u,mag_ab_g,magerr_ab_g,mag_ab_r,magerr_ab_r,mag_ab_i,magerr_ab_i,mag_ab_z,magerr_ab_z,no_sdss_phot,snr,spec_avail
0,74698071455360,3184,54850,476,318454850476,44.726458,0.511273,-77.832683,0.248792,15.265021,0.213877,547.374558,1.266768,18.841656,285.832518,12.364519,19.198263,324.691721,19.068470,18.469217,1.115368,176.115100,-48.741906,22.0,5966.81,400.80,8.437169,0.188926,0.871187,0.124056,0.009347,0.002139,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.254330,67.30388,68.176254,False,1.0,1.0,19.94737,19.18315,18.83133,18.72638,18.69560,0.033923,0.014237,0.015885,0.018644,0.031151,10.494830,21.219350,29.340040,32.31742,33.231230,9.293751,12.915950,5.426850,3.246761,1.097888,19.907561,0.035367,19.183170,0.017398,18.831348,0.018770,18.741408,0.021156,18.726134,0.032716,False,20.964830,2.0
1,74698071455360,708,52175,627,70852175627,44.726458,0.511273,-77.832683,0.248792,15.265021,0.213877,547.374558,1.266768,18.841656,285.832518,12.364519,19.198263,324.691721,19.068470,18.469217,1.115368,176.115100,-48.741906,11.0,5966.81,400.80,8.437169,0.188926,0.871187,0.124056,0.009347,0.002139,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.254330,67.30388,68.176254,False,1.0,1.0,19.94737,19.18315,18.83133,18.72638,18.69560,0.033923,0.014237,0.015885,0.018644,0.031151,10.494830,21.219350,29.340040,32.31742,33.231230,9.293751,12.915950,5.426850,3.246761,1.097888,19.907561,0.035367,19.183170,0.017398,18.831348,0.018770,18.741408,0.021156,18.726134,0.032716,False,11.154221,0.0
2,74698071455360,709,52205,421,70952205421,44.726458,0.511273,-77.832683,0.248792,15.265021,0.213877,547.374558,1.266768,18.841656,285.832518,12.364519,19.198263,324.691721,19.068470,18.469217,1.115368,176.115100,-48.741906,11.0,5966.81,400.80,8.437169,0.188926,0.871187,0.124056,0.009347,0.002139,False,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.254330,67.30388,68.176254,False,1.0,1.0,19.94737,19.18315,18.83133,18.72638,18.69560,0.033923,0.014237,0.015885,0.018644,0.031151,10.494830,21.219350,29.340040,32.31742,33.231230,9.293751,12.915950,5.426850,3.246761,1.097888,19.907561,0.035367,19.183170,0.017398,18.831348,0.018770,18.741408,0.021156,18.726134,0.032716,False,10.822907,0.0
3,74698071455360,4242,55476,963,424255476963,44.726458,0.511273,-77.832683,0.248792,15.265021,0.213877,547.374558,1.266768,18.841656,285.832518,12.364519,19.198263,324.691721,19.068470,18.469217,1.115368,176.115100,-48.741906,23.0,5966.81,400.80,8.437169,0.188926,0.871187,0.124056,0.009347,0.002139,False,dr14,44.726800,0.511211,0.000000,5602.0,65.0,7.890,0.145,5603.0,65.0,7.889,0.145,0.0,0.0,0.52795,0.04949,0.000,0.000,0.013662,0.002369,0.000000,NaN,66.254330,67.30388,68.176254,False,1.0,1.0,19.94737,19.18315,18.83133,18.72638,18.69560,0.033923,0.014237,0.015885,0.018644,0.031151,10.494830,21.219350,29.340040,

### Number of unique WDs in eSDSS:

In [54]:
len(match_tab['gaia_dr3_source_id'].unique())

19257

# Make smaller datasets and save as .csv


In [55]:
#make list of all unique object ids
eSDSS_obj_ids=match_tab['gaia_dr3_source_id'].unique()
np.savetxt('csv/eSDSS_obj_ids.txt', eSDSS_obj_ids.astype(str), fmt="%s")
#save to backup
np.savetxt('csv/eSDSS_obj_ids_nb02.txt', eSDSS_obj_ids.astype(str), fmt="%s")

#break up object ids into sets of 500
#save each set of ids
set_size=400
num_id_sets=int(len(eSDSS_obj_ids)/set_size)+1
print(num_id_sets)
np.savetxt('csv/eSDSS_num_id_sets.txt', np.array([num_id_sets]), fmt="%s")
id_eSDSS_dict={}
for i in tqdm(range(num_id_sets)):
    id_eSDSS_dict['set_'+str(i)]=eSDSS_obj_ids[set_size*i:set_size*i+set_size]
    #save list
    np.savetxt('csv/eSDSS_short_sets/eSDSS_obj_ids_'+str(i)+'.txt', id_eSDSS_dict['set_'+str(i)].astype(str), fmt="%s")
    #save to backup
    np.savetxt('csv/eSDSS_short_sets/eSDSS_obj_ids_'+str(i)+'_nb02.txt', id_eSDSS_dict['set_'+str(i)].astype(str), fmt="%s")
    

49


100%|█████████████████████████████████████████| 49/49 [00:00<00:00, 1051.37it/s]


In [56]:
#make a .csv for each set of ids
for i in range(num_id_sets):
    obj_ids=id_eSDSS_dict['set_'+str(i)]
    #search dataframe for objects in obj_id list
    short_set=match_tab[match_tab['gaia_dr3_source_id'].isin(obj_ids)]
    short_set.to_csv('csv/eSDSS_short_sets/set_'+str(i)+'.csv',index=False)
    #save to backup
    short_set.to_csv('csv/eSDSS_short_sets/set_'+str(i)+'_nb02.csv',index=False)

### Import back in if needed

In [57]:
#import back in the data, either from the backup file or from the latest version of the table
backup=False
if backup==True:
    #read in all object id lists
    eSDSS_num_id_sets=np.genfromtxt('csv/eSDSS_num_id_sets_nb02.txt',dtype=int)
    id_eSDSS_dict={}
    for i in tqdm(range(eSDSS_num_id_sets)):
        id_eSDSS_dict['set_'+str(i)]=np.genfromtxt('csv/eSDSS_short_sets/eSDSS_obj_ids_'+str(i)+'_nb02.txt',dtype=str)
    
    #read in all data tables
    eSDSS_dict={}
    for i in tqdm(range(eSDSS_num_id_sets)):
        eSDSS_dict['set_'+str(i)]=pd.read_csv('csv/eSDSS_short_sets/set_'+str(i)+'_nb02.csv')
else:
    #read in all object id lists
    eSDSS_num_id_sets=np.genfromtxt('csv/eSDSS_num_id_sets.txt',dtype=int)
    id_eSDSS_dict={}
    for i in tqdm(range(eSDSS_num_id_sets)):
        id_eSDSS_dict['set_'+str(i)]=np.genfromtxt('csv/eSDSS_short_sets/eSDSS_obj_ids_'+str(i)+'.txt',dtype=str)
    
    #read in all data tables
    eSDSS_dict={}
    for i in tqdm(range(eSDSS_num_id_sets)):
        eSDSS_dict['set_'+str(i)]=pd.read_csv('csv/eSDSS_short_sets/set_'+str(i)+'.csv')


100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 244.61it/s]
